# BASELINE

In [ ]:
import os, re, json, time, hashlib
from pathlib import Path

os.environ["TRANSFORMERS_NO_APEX"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
import pandas as pd
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

# ---------- CONFIG ----------
TEST_PATH     = Path("test_20.parquet")                 
OUTPUT_JSONL  = Path("BASELINE_llama32_test20.jsonl")
MODEL_NAME    = "meta-llama/Llama-3.2-3B-Instruct"

# concurrency / quality knobs
BATCH_SIZE_REVIEW   = 2     # generate fewer at once for quality (1–2 recommended)
MAX_NEW_TOKENS      = 220
MIN_NEW_TOKENS      = 80
TEMPERATURE         = 0.7
TOP_P               = 0.9
REPETITION_PEN      = 1.15
NO_REPEAT_NGRAM     = 6
DO_SAMPLE           = True  # light sampling improves fluency

# decision gen (deterministic, single-item)
DECISION_MAX_NEW    = 3     # allow a couple tokens; "Accept"/"Reject" may be multi-token
DECISION_TRIES      = 2     # ask at most twice if not exactly Accept/Reject

# ---------- helpers ----------
def parse_prompt_to_title_abs(p: str):
    try:
        title, abstract = p.split("\n\n", 1)
    except Exception:
        title, abstract = p, ""
    return title.strip(), abstract.strip()

def rid_from(title: str, abstract: str):
    h = hashlib.sha1(); h.update(title.encode()); h.update(b"\0"); h.update(abstract.encode()); return h.hexdigest()

def chunked(seq, n):
    for i in range(0, len(seq), n):
        yield seq[i:i+n]

# concise, constrained format to reduce rambling
def make_review_prompt(title, abstract):
    return (
        "You are an ICLR peer reviewer. Write a concise review following EXACTLY this format:\n\n"
        "Decision: <A binary decision between Accept or Reject>\n\n"
        "Comment:\n"
        "- few sentences to justiify the paper deision and balancing its pros/cons. Avoid repetition.\n\n"
        "Strengths:\n"
        "- <strength #1>\n"
        "- <strength #2>\n\n"
        "Weaknesses:\n"
        "- <weakness #1>\n"
        "- <weakness #2>\n\n"
        f"TITLE:\n{title}\n\nABSTRACT:\n{abstract}\n\n"
        "Now produce the review."
    )

def make_decision_prompt(title, abstract):
    # single-word answer; the model must choose one of the two
    return (
        f"TITLE:\n{title}\n\nABSTRACT:\n{abstract}\n\n"
        "Output exactly one of the following words and nothing else:\n"
        "Accept\nReject\n\n"
        "Answer:"
    )

def extract_accept_reject(s: str):
    m = re.search(r"\b(Accept|Reject)\b", s, re.I)
    return m.group(1).title() if m else None

# ---------- load test & group by paper ----------
assert TEST_PATH.exists(), f"Missing {TEST_PATH}"
df = pd.read_parquet(TEST_PATH)
assert "prompt" in df.columns, "Expected a 'prompt' column."

groups = {}
for _, row in df.iterrows():
    title, abstract = parse_prompt_to_title_abs(str(row["prompt"]))
    rid = rid_from(title, abstract)
    if rid not in groups:
        groups[rid] = (title, abstract)

papers = list(groups.keys())
print(f"Papers (unique title+abstract): {len(papers)}  |  Total rows: {len(df)}")

# ---------- resume support ----------
done = set()
if OUTPUT_JSONL.exists():
    with open(OUTPUT_JSONL, "r", encoding="utf-8") as f:
        for line in f:
            try:
                j = json.loads(line)
                done.add(j["rid"])
            except:
                pass
todo = [rid for rid in papers if rid not in done]
print(f"Already done: {len(done)}  |  To generate now: {len(todo)}")

# ---------- load BASE model ----------
torch.backends.cuda.matmul.allow_tf32 = True
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype  = torch.bfloat16 if device == "cuda" else torch.float32
print("CUDA available:", torch.cuda.is_available())

tok = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=dtype,
    trust_remote_code=True,
    device_map="auto",
).eval()
model.config.use_cache = True

eos_ids = tok.eos_token_id if isinstance(tok.eos_token_id, (list,tuple)) else [tok.eos_token_id]
pad_id  = tok.pad_token_id

review_gen_kwargs = dict(
    max_new_tokens=MAX_NEW_TOKENS,
    min_new_tokens=MIN_NEW_TOKENS,
    do_sample=DO_SAMPLE,
    temperature=TEMPERATURE,
    top_p=TOP_P,
    repetition_penalty=REPETITION_PEN,
    no_repeat_ngram_size=NO_REPEAT_NGRAM,
    eos_token_id=eos_ids,
    pad_token_id=pad_id,
    return_dict_in_generate=True,
)

# deterministic, tiny decision call
def generate_binary_decision(title, abstract):
    prompt = make_decision_prompt(title, abstract)
    for attempt in range(DECISION_TRIES):
        enc = tok(prompt, return_tensors="pt", truncation=True).to(model.device)
        with torch.inference_mode():
            out = model.generate(
                **enc,
                max_new_tokens=DECISION_MAX_NEW,
                do_sample=False,
                eos_token_id=eos_ids,
                pad_token_id=pad_id,
                return_dict_in_generate=True,
            )
        tail = out.sequences[0, enc["attention_mask"].sum().item():]
        txt = tok.decode(tail, skip_special_tokens=True).strip()
        dec = extract_accept_reject(txt)
        if dec in ("Accept","Reject"):
            return dec
        
        prompt = (
            f"TITLE:\n{title}\n\nABSTRACT:\n{abstract}\n\n"
            "Return exactly one word, either Accept or Reject. No other text.\n"
            "Answer:"
        )
    return None

# warm-up
_warm = tok("Say 'ready' and stop.", return_tensors="pt").to(model.device)
with torch.inference_mode():
    _ = model.generate(**_warm, max_new_tokens=4, do_sample=False)
print("🔥 Warm-up done.")

# ---------- main loop ----------
start = time.time()
written = 0
with open(OUTPUT_JSONL, "a", encoding="utf-8") as fout:
    pbar = tqdm(total=len(todo), desc="⚡ Generating review+decision (BASE)", unit="paper")
    for batch_rids in chunked(todo, BATCH_SIZE_REVIEW):
        titles    = [groups[r][0] for r in batch_rids]
        abstracts = [groups[r][1] for r in batch_rids]
        prompts   = [make_review_prompt(t, a) for t, a in zip(titles, abstracts)]

        enc = tok(prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)
        input_lens = enc["attention_mask"].sum(dim=1).tolist()

        with torch.inference_mode():
            out = model.generate(
                input_ids=enc["input_ids"],
                attention_mask=enc["attention_mask"],
                **review_gen_kwargs,
            )

        seqs  = out.sequences
        tails = [seqs[i, input_lens[i]:] for i in range(seqs.size(0))]
        reviews = tok.batch_decode(tails, skip_special_tokens=True)

        # strict decision per item (separate micro-call; deterministic)
        for rid, t, a, review in zip(batch_rids, titles, abstracts, reviews):
            decision = generate_binary_decision(t, a)
            rec = {
                "rid": rid,
                "title": t,
                "abstract": a,
                "review": review.strip(),
                "decision": decision  # 'Accept' or 'Reject' (or None if the model somehow failed twice)
            }
            fout.write(json.dumps(rec, ensure_ascii=False) + "\n")
        fout.flush()

        written += len(batch_rids)
        pbar.update(len(batch_rids))
    pbar.close()

mins = (time.time() - start) / 60
print(f"✅ Done. Wrote {written} items to {OUTPUT_JSONL} in {mins:.1f} min")


Papers (unique title+abstract): 2203  |  Total rows: 6691
Already done: 0  |  To generate now: 2203
CUDA available: True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


🔥 Warm-up done.


⚡ Generating review+decision (BASE):   0%|          | 0/2203 [00:00<?, ?paper/s]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

✅ Done. Wrote 2203 items to BASELINE_llama32_test20.jsonl in 91.8 min


# Finetuning with Lora

In [ ]:
import os, re, json, math, time, random, gc
from pathlib import Path
os.environ["TRANSFORMERS_NO_APEX"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
torch.backends.cuda.matmul.allow_tf32 = True

import pandas as pd
from datasets import Dataset
from tqdm.auto import tqdm

from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    Trainer, TrainingArguments, BitsAndBytesConfig,
)
from transformers.trainer_callback import EarlyStoppingCallback
from peft import (
    LoraConfig, get_peft_model, prepare_model_for_kbit_training,
)

# ---- File paths ----
DATA_PATH   = Path("train_17_18_19.parquet")      # input parquet with columns
OUTPUT_DIR  = Path("lora_json_outputs_es")        # where LoRA adapters + logs go
MODEL_NAME  = "meta-llama/Llama-3.2-3B-Instruct"

# ---- Training knobs (safe for A40 48GB, 4-bit) ----
MAX_LEN               = 1408        
LR                    = 1e-4
BATCH_SIZE_PER_DEVICE = 4
GRAD_ACCUM_STEPS      = 8           
WARMUP_RATIO          = 0.03
WEIGHT_DECAY          = 0.1
TRAIN_EPOCHS_MAX      = 10          
EVAL_STEPS            = 200
SAVE_STEPS            = 200
LOG_STEPS             = 50
EARLY_STOP_PATIENCE   = 3
EARLY_STOP_THRESHOLD  = 0.0
SEED                  = 42

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype  = torch.bfloat16 if device == "cuda" else torch.float32
print("CUDA available:", torch.cuda.is_available())

# ----------------- 1) Load data -----------------
assert DATA_PATH.exists(), f"Missing file: {DATA_PATH}"
raw_df = pd.read_parquet(DATA_PATH)
need_cols = {"prompt","completion","year"}
missing = need_cols - set(raw_df.columns)
assert not missing, f"Dataset missing columns: {missing}"

# Split by year: <=2020 train, 2021 val (fallback to small holdout if no 2021)
train_df = raw_df[raw_df["year"] <= 2020].copy()
eval_df  = raw_df[raw_df["year"] == 2021].copy()
if len(eval_df) == 0:
    frac = 0.08
    eval_df = train_df.sample(frac=frac, random_state=SEED)
    train_df = train_df.drop(eval_df.index)
    print(f"⚠️ No 2021 rows. Using {len(eval_df)} ({int(frac*100)}%) of train as validation.")
else:
    print(f"✅ Using 2021 rows as validation: {len(eval_df)}")
train_df = train_df.reset_index(drop=True)
eval_df  = eval_df.reset_index(drop=True)
print(f"Train rows: {len(train_df)} | Eval rows: {len(eval_df)}")

# ----------------- 2) Build strict-JSON targets (heuristics) -----------------
# We convert each free-form 'completion' into the exact JSON schema we want the model to emit.
POS_CUES = {"strong","novel","well","clear","solid","comprehensive","convincing","significant","improve","robust"}
NEG_CUES = {"weak","limited","unclear","missing","lacking","concern","issue","fail","noisy","incomplete","overfit","bias"}

def extract_decision(text: str):
    m = re.search(r"\bDecision\s*[:\-]?\s*(Accept|Reject)\b", text, re.I)
    return m.group(1).title() if m else None

def sentence_split(text: str):
    # simple splitter
    parts = re.split(r'(?<=[\.\?\!])\s+', text.strip())
    return [s.strip() for s in parts if s.strip()]

def bullets_from_text(text: str):
    lines = [l.strip(" -*+\t") for l in text.splitlines() if l.strip()]
    return [l for l in lines if len(l.split()) >= 3]

def pick_items(candidates, cues, k=2):
    scored = []
    for s in candidates:
        score = sum(1 for w in cues if re.search(rf"\b{re.escape(w)}\b", s, re.I))
        scored.append((score, -len(s), s))  # prefer cue hits, then shorter
    scored.sort(reverse=True)
    picked = [s for _,__,s in scored[:k]]
    # pad if needed with other candidates
    if len(picked) < k:
        for s in candidates:
            if s not in picked and len(picked) < k:
                picked.append(s)
    return picked[:k]

def to_target_json(review_text: str):
    # remove the decision line from the body
    dec = extract_decision(review_text)
    body = re.sub(r"\bDecision\s*[:\-]?\s*(Accept|Reject)\b.*", "", review_text, flags=re.I|re.S).strip()

    # candidates from bullets and sentences
    bullets = bullets_from_text(body)
    sents   = sentence_split(body)
    cands   = bullets if len(bullets) >= 4 else sents

    strengths = pick_items(cands, POS_CUES, k=2)
    # avoid duplicates between lists
    remaining = [c for c in cands if c not in strengths]
    weaknesses = pick_items(remaining, NEG_CUES, k=2)

    # comment = 2-4 short sentences
    comment_sents = [s for s in sents if s not in strengths and s not in weaknesses]
    comment = " ".join(comment_sents[:3]).strip()
    if len(comment) < 40:  # ensure something substantive
        comment = " ".join(sents[:3]).strip()

    # if decision missing, we can infer weakly from sentiment, else drop row
    if dec is None:
        # naive heuristic: more NEG hits → Reject
        pos_hits = sum(any(re.search(rf"\b{w}\b", s, re.I) for w in POS_CUES) for s in strengths)
        neg_hits = sum(any(re.search(rf"\b{w}\b", s, re.I) for w in NEG_CUES) for s in weaknesses)
        dec = "Reject" if neg_hits >= pos_hits else "Accept"

    # enforce exactly two items, fallback to generic if we couldn't find enough
    def fill(lst, filler):
        lst = [x for x in lst if x]  # remove empties
        while len(lst) < 2:
            lst.append(filler)
        return lst[:2]

    strengths = fill(strengths, "Clear motivation and empirical support.")
    weaknesses = fill(weaknesses, "Limited analysis and missing ablations.")

    target = {
        "comment": comment[:1200],  # cap to keep JSON small
        "strengths": [s[:400] for s in strengths],
        "weaknesses": [w[:400] for w in weaknesses],
        "decision": dec if dec in ("Accept","Reject") else "Reject",
    }
    return target

def build_json_rows(df: pd.DataFrame):
    rows = []
    drops = 0
    for _, r in df.iterrows():
        prompt = r["prompt"]
        # prompt is already "TITLE\n\nABSTRACT" 
        try:
            title, abstract = prompt.split("\n\n", 1)
        except Exception:
            title, abstract = prompt, ""
        tgt = to_target_json(str(r["completion"]))
        # prompt we train on (instructs exact JSON)
        sys_prompt = (
            "You are an ICLR peer reviewer. Return ONLY valid JSON with keys: "
            'comment, strengths, weaknesses, decision. '
            'Exactly two strengths and two weaknesses. Decision must be "Accept" or "Reject".'
        )
        train_prompt = (
            f"{sys_prompt}\n\nTITLE:\n{title}\n\nABSTRACT:\n{abstract}\n\n"
            "Respond with JSON only."
        )
        rows.append({
            "prompt": train_prompt,
            "target_json": json.dumps(tgt, ensure_ascii=False),
        })
    print(f"Built {len(rows)} JSON rows (dropped {drops}).")
    return pd.DataFrame(rows)

train_json_df = build_json_rows(train_df)
eval_json_df  = build_json_rows(eval_df)

# ----------------- 3) Tokenizer & masked tokenization -----------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tok_mask_batch(batch):
    in_ids, labels, attn = [], [], []
    for prompt, target_json in zip(batch["prompt"], batch["target_json"]):
        p_ids = tokenizer(prompt, add_special_tokens=False)["input_ids"]
        y_ids = tokenizer(target_json + tokenizer.eos_token, add_special_tokens=False)["input_ids"]

        max_prompt = MAX_LEN - len(y_ids)
        if max_prompt < 0:
            # ensure we always keep full target JSON
            y_ids = y_ids[-(MAX_LEN-1):] + [tokenizer.eos_token_id]
            p_ids = []
        else:
            p_ids = p_ids[-max_prompt:]

        ids = p_ids + y_ids
        lab = [-100]*len(p_ids) + y_ids
        am  = [1]*len(ids)

        pad = MAX_LEN - len(ids)
        if pad > 0:
            ids += [tokenizer.pad_token_id]*pad
            lab += [-100]*pad
            am  += [0]*pad

        in_ids.append(ids); labels.append(lab); attn.append(am)
    return {"input_ids": in_ids, "labels": labels, "attention_mask": attn}

train_ds = Dataset.from_pandas(train_json_df).map(
    tok_mask_batch, batched=True, remove_columns=["prompt","target_json"]
)
eval_ds  = Dataset.from_pandas(eval_json_df).map(
    tok_mask_batch, batched=True, remove_columns=["prompt","target_json"]
)
print("Tokenized:", len(train_ds), len(eval_ds), train_ds.column_names)

# ----------------- 4) Model + LoRA (bnb 4-bit) -----------------
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if device=="cuda" else torch.float32,
)

base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True,
)
base = prepare_model_for_kbit_training(base)

lora_cfg = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05, bias="none",
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    task_type="CAUSAL_LM",
)
model = get_peft_model(base, lora_cfg)
model.gradient_checkpointing_enable()  # will set use_cache=False internally

# ----------------- 5) Train (Early Stopping) -----------------
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
args = TrainingArguments(
    output_dir=str(OUTPUT_DIR),
    num_train_epochs=TRAIN_EPOCHS_MAX,
    per_device_train_batch_size=BATCH_SIZE_PER_DEVICE,
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    learning_rate=LR,
    warmup_ratio=WARMUP_RATIO,
    weight_decay=WEIGHT_DECAY,
    lr_scheduler_type="cosine",
    logging_steps=LOG_STEPS,
    save_strategy="steps",
    save_steps=SAVE_STEPS,
    eval_strategy="steps",
    eval_steps=EVAL_STEPS,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    bf16=(device=="cuda"),
    report_to="none",
    save_total_limit=3,
    seed=SEED,
    remove_unused_columns=False,   # keep our columns
)

callbacks = [EarlyStoppingCallback(
    early_stopping_patience=EARLY_STOP_PATIENCE,
    early_stopping_threshold=EARLY_STOP_THRESHOLD
)]

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    callbacks=callbacks,
)

print("🚀 Training (JSON target)…")
trainer.train()
print("✅ Training done. Best:", trainer.state.best_model_checkpoint)

trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
metrics = trainer.evaluate()
if "eval_loss" in metrics:
    print(f"Eval loss: {metrics['eval_loss']:.4f} | Perplexity: {math.exp(metrics['eval_loss']):.2f}")

# ----------------- 6) Quick sanity check: generate & validate JSON -----------------
from peft import PeftModel
from transformers import AutoModelForCausalLM

def force_json_block(text: str):
    # try to extract first {...} block
    m = re.search(r'\{.*\}', text, re.S)
    return m.group(0) if m else text

def is_valid_schema(j):
    return (
        isinstance(j, dict)
        and set(j.keys()) == {"comment","strengths","weaknesses","decision"}
        and isinstance(j["comment"], str)
        and isinstance(j["strengths"], list) and len(j["strengths"]) == 2
        and isinstance(j["weaknesses"], list) and len(j["weaknesses"]) == 2
        and j["decision"] in ("Accept","Reject")
    )

# reload best model for generation
del trainer, model, base
gc.collect(); 
if torch.cuda.is_available(): torch.cuda.empty_cache()

gen_base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, quantization_config=quant_config, device_map="auto", trust_remote_code=True
)
gen_model = PeftModel.from_pretrained(gen_base, OUTPUT_DIR).eval()
tok = tokenizer

def make_prompt(title, abstract):
    return (
        "You are an ICLR peer reviewer. Return ONLY valid JSON with keys: "
        'comment, strengths, weaknesses, decision. Exactly two strengths and two weaknesses. '
        'Decision must be "Accept" or "Reject".\n\n'
        f"TITLE:\n{title}\n\nABSTRACT:\n{abstract}\n\nRespond with JSON only."
    )

# test on 3 random samples from eval_df
sampled = eval_df.sample(n=min(3, len(eval_df)), random_state=SEED)
for _, r in sampled.iterrows():
    try:
        title, abstract = r["prompt"].split("\n\n", 1)
    except Exception:
        title, abstract = r["prompt"], ""
    prompt = make_prompt(title, abstract)
    enc = tok(prompt, return_tensors="pt", truncation=True, padding=False).to(gen_model.device)
    with torch.inference_mode():
        out = gen_model.generate(
            **enc,
            max_new_tokens=220,
            do_sample=False,  # deterministic for sanity check
            eos_token_id=tok.eos_token_id[0] if isinstance(tok.eos_token_id,(list,tuple)) else tok.eos_token_id,
            pad_token_id=tok.pad_token_id,
            return_dict_in_generate=True,
        )
    full = tok.decode(out.sequences[0], skip_special_tokens=True)
    gen_only = full[len(prompt):].strip()
    blob = force_json_block(gen_only).strip()
    ok, parsed = False, None
    try:
        parsed = json.loads(blob)
        ok = is_valid_schema(parsed)
    except Exception:
        ok = False
    print("\n--- SAMPLE ---")
    print("RAW:", gen_only[:300], "...")
    print("JSON OK:", ok)
    if ok:
        print(parsed)

CUDA available: True
⚠️ No 2021 rows. Using 723 (8%) of train as validation.
Train rows: 8320 | Eval rows: 723
Built 8320 JSON rows (dropped 0).
Built 723 JSON rows (dropped 0).


Map:   0%|          | 0/8320 [00:00<?, ? examples/s]

Map:   0%|          | 0/723 [00:00<?, ? examples/s]

Tokenized: 8320 723 ['input_ids', 'labels', 'attention_mask']


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

✅ Training done. Best: lora_json_outputs_es/checkpoint-400


Eval loss: 2.2727 | Perplexity: 9.71


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/mobs/.local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/mobs/.local/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.



--- SAMPLE ---
RAW: {"comment": "This paper presents a theoretical interpretation of RAML, which is a simple and effective learning framework to directly optimize towards the reward function in structured prediction tasks. The authors introduce softmax Q-distribution estimation, a novel theoretical interpretation of RA ...
JSON OK: False

--- SAMPLE ---
RAW: {"comment": "This paper proposes a new model for learning disentangled representations of data. The model is based on a variational autoencoder framework, and is able to learn representations across data sets exhibiting a number of distinct contexts. The model is able to combine these representation ...
JSON OK: True
{'comment': 'This paper proposes a new model for learning disentangled representations of data. The model is based on a variational autoencoder framework, and is able to learn representations across data sets exhibiting a number of distinct contexts. The model is able to combine these representations to generate data 

### Testing the trained model

In [13]:
import os, re, json, time, hashlib
from pathlib import Path

os.environ["TRANSFORMERS_NO_APEX"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
import pandas as pd
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# ---------- CONFIG ----------
TEST_PATH     = Path("test_20.parquet")           
OUTPUT_JSONL  = Path("TRAINED_llama32_test20_raw.jsonl")   # raw generations per paper
MODEL_NAME    = "meta-llama/Llama-3.2-3B-Instruct"
ADAPTER_DIR   = None   # e.g., "lora_json_outputs_es/checkpoint-400"; None = auto-pick

BATCH_SIZE       = 4
MAX_NEW_TOKENS   = 220
MIN_NEW_TOKENS   = 80
TEMPERATURE      = 0.7
TOP_P            = 0.9
REPETITION_PEN   = 1.15
NO_REPEAT_NGRAM  = 6
DO_SAMPLE        = True  # small sampling often reduces repetition vs pure greedy

# ---------- Helpers ----------
def pick_best_adapter():
    for root in ["lora_json_outputs_es", "lora_outputs_es", "lora_json_outputs", "lora_outputs"]:
        st = Path(root) / "trainer_state.json"
        if st.exists():
            try:
                best = json.loads(st.read_text()).get("best_model_checkpoint")
                if best and Path(best).exists():
                    return best
            except Exception:
                pass
        if Path(root).exists():
            return root
    raise FileNotFoundError("No adapter folder found. Set ADAPTER_DIR manually.")

def parse_prompt_to_title_abs(p: str):
    try:
        title, abstract = p.split("\n\n", 1)
    except Exception:
        title, abstract = p, ""
    return title.strip(), abstract.strip()

def rid_from(title: str, abstract: str):
    h = hashlib.sha1(); h.update(title.encode()); h.update(b"\0"); h.update(abstract.encode()); return h.hexdigest()

def chunked(seq, n):
    for i in range(0, len(seq), n):
        yield seq[i:i+n]

def has_decision(txt: str) -> bool:
    return re.search(r"\bDecision\s*:\s*(Accept|Reject)\b", txt, flags=re.I) is not None

# Strong, explicit format to reduce rambling and force a final decision
def make_prompt(title, abstract):
    return (
        "You are an ICLR peer reviewer. Write a concise review following EXACTLY this format:\n\n"
        "Decision: <A binary decision between Accept or Reject>\n\n"
        "Comment:\n"
        "- few sentences to justiify the paper deision and balancing its pros/cons. Avoid repetition.\n\n"
        "Strengths:\n"
        "- <strength #1>\n"
        "- <strength #2>\n\n"
        "Weaknesses:\n"
        "- <weakness #1>\n"
        "- <weakness #2>\n\n"
        f"TITLE:\n{title}\n\nABSTRACT:\n{abstract}\n\n"
        "Now produce the review."
    )

# ---------- Load test & group by paper ----------
assert TEST_PATH.exists(), f"Missing {TEST_PATH}"
df = pd.read_parquet(TEST_PATH)
assert "prompt" in df.columns, "Expected a 'prompt' column."

groups = {}
for _, row in df.iterrows():
    title, abstract = parse_prompt_to_title_abs(str(row["prompt"]))
    rid = rid_from(title, abstract)
    if rid not in groups:
        groups[rid] = (title, abstract)

papers = list(groups.keys())
print(f"Papers (unique title+abstract): {len(papers)}  |  Total rows in file: {len(df)}")

# ---------- Resume support ----------
done = set()
if OUTPUT_JSONL.exists():
    with open(OUTPUT_JSONL, "r", encoding="utf-8") as f:
        for line in f:
            try:
                j = json.loads(line)
                done.add(j["rid"])
            except:
                pass

todo = [rid for rid in papers if rid not in done]
print(f"Already done: {len(done)}  |  To generate now: {len(todo)}")

# ---------- Load base + adapters ----------
if ADAPTER_DIR is None:
    ADAPTER_DIR = pick_best_adapter()
print("Using adapters from:", ADAPTER_DIR)

torch.backends.cuda.matmul.allow_tf32 = True
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype  = torch.bfloat16 if device == "cuda" else torch.float32
print("CUDA available:", torch.cuda.is_available())

tok = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=dtype,
    trust_remote_code=True,
    device_map="auto",
)
model = PeftModel.from_pretrained(base, ADAPTER_DIR).eval()
model.config.use_cache = True

eos_ids = tok.eos_token_id if isinstance(tok.eos_token_id, (list,tuple)) else [tok.eos_token_id]
pad_id  = tok.pad_token_id

gen_kwargs = dict(
    max_new_tokens=MAX_NEW_TOKENS,
    min_new_tokens=MIN_NEW_TOKENS,
    do_sample=DO_SAMPLE,
    temperature=TEMPERATURE,
    top_p=TOP_P,
    repetition_penalty=REPETITION_PEN,
    no_repeat_ngram_size=NO_REPEAT_NGRAM,
    eos_token_id=eos_ids,
    pad_token_id=pad_id,
    return_dict_in_generate=True,
)

# Tiny fallback to add a decision if missing
def generate_decision_only(title, abstract):
    prompt = (
        f"TITLE:\n{title}\n\nABSTRACT:\n{abstract}\n\n"
        "Given the paper above, output exactly one line with your final verdict:\n"
        "Decision: Accept\n"
        "or\n"
        "Decision: Reject"
    )
    enc = tok(prompt, return_tensors="pt", truncation=True).to(model.device)
    with torch.inference_mode():
        out = model.generate(
            **enc, max_new_tokens=6, do_sample=False,
            eos_token_id=eos_ids, pad_token_id=pad_id, return_dict_in_generate=True
        )
    tail = out.sequences[0, enc["attention_mask"].sum().item():]
    return tok.decode(tail, skip_special_tokens=True).strip()

# Warm-up
_warm = tok("Say 'ready' and stop.", return_tensors="pt").to(model.device)
with torch.inference_mode():
    _ = model.generate(**_warm, max_new_tokens=4, do_sample=False)
print("🔥 Warm-up done.")

# ---------- Inference loop ----------
start = time.time()
written = 0
with open(OUTPUT_JSONL, "a", encoding="utf-8") as fout:
    pbar = tqdm(total=len(todo), desc="⚡ Generating (quality)", unit="paper")
    for batch_rids in chunked(todo, BATCH_SIZE):
        titles    = [groups[r][0] for r in batch_rids]
        abstracts = [groups[r][1] for r in batch_rids]
        prompts   = [make_prompt(t, a) for t, a in zip(titles, abstracts)]

        enc = tok(prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)
        input_lens = enc["attention_mask"].sum(dim=1).tolist()

        with torch.inference_mode():
            out = model.generate(
                input_ids=enc["input_ids"],
                attention_mask=enc["attention_mask"],
                **gen_kwargs,
            )

        seqs  = out.sequences
        tails = [seqs[i, input_lens[i]:] for i in range(seqs.size(0))]
        raws  = tok.batch_decode(tails, skip_special_tokens=True)

        # Fallback: add Decision line if missing
        for i, raw in enumerate(raws):
            if not has_decision(raw):
                try:
                    dec_line = generate_decision_only(titles[i], abstracts[i])
                    if "Decision:" in dec_line:
                        raw = raw.rstrip() + ("\n\n" if not raw.endswith("\n") else "") + dec_line
                        raws[i] = raw
                except Exception:
                    pass

        for rid, title, abstract, raw in zip(batch_rids, titles, abstracts, raws):
            rec = {"rid": rid, "title": title, "abstract": abstract, "raw": raw.strip()}
            fout.write(json.dumps(rec, ensure_ascii=False) + "\n")
        fout.flush()

        written += len(batch_rids)
        pbar.update(len(batch_rids))
    pbar.close()

mins = (time.time() - start) / 60
print(f"✅ Done. Wrote {written} RAW generations to {OUTPUT_JSONL} in {mins:.1f} min")

Papers (unique title+abstract): 2203  |  Total rows in file: 6691
Already done: 0  |  To generate now: 2203
Using adapters from: lora_json_outputs_es
CUDA available: True


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


🔥 Warm-up done.


⚡ Generating (quality):   0%|          | 0/2203 [00:00<?, ?paper/s]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

✅ Done. Wrote 2203 RAW generations to llama32_test20_raw_8.jsonl in 88.3 min


# Evaluation

### Normalization of the outputs

In [ ]:
# === CONFIG ===
INPUT_PATH  = "BASELINE_llama32_test20.jsonl"               # raw generations JSONL
OUTPUT_PATH = "BASELINE_llama32_test20_NORM.jsonl" # same records + parsed fields
STRICT_BINARY_ONLY = True  

import json, re
from pathlib import Path
from collections import Counter

def _clean(s: str) -> str:
    if not isinstance(s, str): return ""
    # strip code fences or extra quotes
    s = s.strip().strip("`").strip()
    return s

def _try_parse_nested_json(s: str):
    """If the string contains a JSON object, try to parse it and return a dict; else None."""
    if not s or not isinstance(s, str): return None
    s = s.strip()
    # whole-string JSON
    if s.startswith("{") and s.endswith("}"):
        try: return json.loads(s)
        except Exception: pass
    # embedded {...} JSON
    l, r = s.find("{"), s.rfind("}")
    if 0 <= l < r:
        try: return json.loads(s[l:r+1])
        except Exception: pass
    return None

def _norm_decision(text: str):
    """Return 'Accept'/'Reject' if seen in text; otherwise None. Choose the last occurrence."""
    if not text: return None
    # explicit heading “Decision: Accept/Reject”
    m = re.search(r"(?im)^\s*#{0,3}\s*decision\s*[:\-#]*\s*(accept|reject)\b", text)
    if m: return m.group(1).title()
    # anywhere “Decision: Accept/Reject”
    m = re.search(r"(?i)\bdecision\s*[:\-]?\s*(accept|reject)\b", text)
    if m: return m.group(1).title()
    # fallback: last bare Accept/Reject in the text
    hits = [h.group(1).lower() for h in re.finditer(r"\b(accept|reject)\b", text, flags=re.I)]
    if hits:
        return "Accept" if hits[-1] == "accept" else "Reject"
    return None

def parse_decision_from_record(rec: dict):
    """Priority:
       1) explicit fields ('decision', 'paper_decision', etc.) if strictly Accept/Reject
       2) nested JSON inside 'raw'/'review' if present
       3) regex scan of strings: 'raw', 'review', 'comment', 'output', 'response', 'completion'
    """
    # 1) direct fields
    for k in ("parsed_decision", "decision", "paper_decision", "pred_decision", "review_decision"):
        v = rec.get(k)
        if isinstance(v, str) and v.strip().lower() in ("accept","reject"):
            return v.title()

    # 2) nested JSON inside text fields
    for k in ("raw", "review", "comment", "output", "response", "completion"):
        v = rec.get(k)
        if isinstance(v, str) and "{" in v and "}" in v:
            j = _try_parse_nested_json(v)
            if isinstance(j, dict):
                d = j.get("decision")
                if isinstance(d, str) and d.strip().lower() in ("accept","reject"):
                    return d.title()

    # 3) regex scanning
    for k in ("raw", "review", "comment", "output", "response", "completion"):
        v = rec.get(k)
        if isinstance(v, str):
            d = _norm_decision(_clean(v))
            if d is not None:
                return d

    return None

# ---- run normalization (decision only) ----
inp, out = Path(INPUT_PATH), Path(OUTPUT_PATH)
assert inp.exists(), f"Input not found: {inp.resolve()}"

total, written = 0, 0
counts = Counter()

with inp.open("r", encoding="utf-8") as fin, out.open("w", encoding="utf-8") as fout:
    for line in fin:
        line = line.strip()
        if not line: continue
        try:
            rec = json.loads(line)
        except Exception:
            continue
        total += 1

        decision = parse_decision_from_record(rec)
        if STRICT_BINARY_ONLY and decision not in ("Accept","Reject"):
            rec["parsed_decision"] = None
            rec["parsed_decision_int"] = None
        else:
            rec["parsed_decision"] = decision
            rec["parsed_decision_int"] = (1 if decision == "Accept" else 0) if decision else None

        fout.write(json.dumps(rec, ensure_ascii=False) + "\n")
        written += 1
        if rec["parsed_decision"] in ("Accept","Reject"):
            counts[rec["parsed_decision"]] += 1

print(f"✅ Wrote {written}/{total} lines → {out.name}")
print(f"   decisions found: {sum(counts.values())}  |  {dict(counts)}")


✅ Wrote 2203/2203 lines → BASELINE_llama32_test20_NORM.jsonl
   decisions found: 2203  |  {'Accept': 2199, 'Reject': 4}


In [1]:
# === CONFIG ===
INPUT_PATH  = "TRAINED_llama32_test20_raw_output.jsonl"               # raw generations JSONL
OUTPUT_PATH = "TRAINED_llama32_test20_raw_output_NORM.jsonl" # same records + parsed fields
STRICT_BINARY_ONLY = True  # keep only exact Accept/Reject; don't map “Weak/Borderline” etc.

import json, re
from pathlib import Path
from collections import Counter

def _clean(s: str) -> str:
    if not isinstance(s, str): return ""
    # strip code fences or extra quotes
    s = s.strip().strip("`").strip()
    return s

def _try_parse_nested_json(s: str):
    """If the string contains a JSON object, try to parse it and return a dict; else None."""
    if not s or not isinstance(s, str): return None
    s = s.strip()
    # whole-string JSON
    if s.startswith("{") and s.endswith("}"):
        try: return json.loads(s)
        except Exception: pass
    # embedded {...} JSON
    l, r = s.find("{"), s.rfind("}")
    if 0 <= l < r:
        try: return json.loads(s[l:r+1])
        except Exception: pass
    return None

def _norm_decision(text: str):
    """Return 'Accept'/'Reject' if seen in text; otherwise None. Choose the last occurrence."""
    if not text: return None
    # explicit heading “Decision: Accept/Reject”
    m = re.search(r"(?im)^\s*#{0,3}\s*decision\s*[:\-#]*\s*(accept|reject)\b", text)
    if m: return m.group(1).title()
    # anywhere “Decision: Accept/Reject”
    m = re.search(r"(?i)\bdecision\s*[:\-]?\s*(accept|reject)\b", text)
    if m: return m.group(1).title()
    # fallback: last bare Accept/Reject in the text
    hits = [h.group(1).lower() for h in re.finditer(r"\b(accept|reject)\b", text, flags=re.I)]
    if hits:
        return "Accept" if hits[-1] == "accept" else "Reject"
    return None

def parse_decision_from_record(rec: dict):
    """Priority:
       1) explicit fields ('decision', 'paper_decision', etc.) if strictly Accept/Reject
       2) nested JSON inside 'raw'/'review' if present
       3) regex scan of strings: 'raw', 'review', 'comment', 'output', 'response', 'completion'
    """
    # 1) direct fields
    for k in ("parsed_decision", "decision", "paper_decision", "pred_decision", "review_decision"):
        v = rec.get(k)
        if isinstance(v, str) and v.strip().lower() in ("accept","reject"):
            return v.title()

    # 2) nested JSON inside text fields
    for k in ("raw", "review", "comment", "output", "response", "completion"):
        v = rec.get(k)
        if isinstance(v, str) and "{" in v and "}" in v:
            j = _try_parse_nested_json(v)
            if isinstance(j, dict):
                d = j.get("decision")
                if isinstance(d, str) and d.strip().lower() in ("accept","reject"):
                    return d.title()

    # 3) regex scanning
    for k in ("raw", "review", "comment", "output", "response", "completion"):
        v = rec.get(k)
        if isinstance(v, str):
            d = _norm_decision(_clean(v))
            if d is not None:
                return d

    return None

# ---- run normalization (decision only) ----
inp, out = Path(INPUT_PATH), Path(OUTPUT_PATH)
assert inp.exists(), f"Input not found: {inp.resolve()}"

total, written = 0, 0
counts = Counter()

with inp.open("r", encoding="utf-8") as fin, out.open("w", encoding="utf-8") as fout:
    for line in fin:
        line = line.strip()
        if not line: continue
        try:
            rec = json.loads(line)
        except Exception:
            continue
        total += 1

        decision = parse_decision_from_record(rec)
        if STRICT_BINARY_ONLY and decision not in ("Accept","Reject"):
            rec["parsed_decision"] = None
            rec["parsed_decision_int"] = None
        else:
            rec["parsed_decision"] = decision
            rec["parsed_decision_int"] = (1 if decision == "Accept" else 0) if decision else None

        fout.write(json.dumps(rec, ensure_ascii=False) + "\n")
        written += 1
        if rec["parsed_decision"] in ("Accept","Reject"):
            counts[rec["parsed_decision"]] += 1

print(f"✅ Wrote {written}/{total} lines → {out.name}")
print(f"   decisions found: {sum(counts.values())}  |  {dict(counts)}")


✅ Wrote 2203/2203 lines → TRAINED_llama32_test20_raw_output_NORM.jsonl
   decisions found: 2196  |  {'Accept': 1650, 'Reject': 546}


In [21]:
!pip install -q sentence-transformers


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.13.0a0+fae8e8c requires torch==1.14.0a0+410ce96, but you have torch 2.1.1 which is incompatible.
thinc 8.1.5 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.10.6 which is incompatible.
spacy 3.4.4 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.10.6 which is incompatible.
spacy 3.4.4 requires typer<0.8.0,>=0.3.0, but you have typer 0.15.2 which is incompatible.
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


### Evaluation Metrics

In [ ]:
# ================== CONFIG ==================
TEST_PARQUET          = "test_2020.parquet"           # has columns: title, abstract, review, paper_decision
BASELINE_JSONL        = "BASELINE_llama32_test20_NORM.jsonl"  #  baseline outputs (normalized or raw)
TRAINED_JSONL         = "TRAINED_llama32_test20_raw_output_NORM.jsonl"   #  trained outputs (normalized or raw)
RESULTS_CSV           = "eval_results_baseline_vs_trained.csv"

PRED_TEXT_KEYS = ["review", "comment", "raw", "output", "response", "completion"]

import re, json, collections
from pathlib import Path
import pandas as pd

# ---------- Normalization helpers tuned to format ----------
def norm_ws(s: str) -> str:
    return re.sub(r"\s+", " ", s).strip()

def strip_prefixes(s: str) -> str:
    s = "" if s is None else str(s)
    # remove leading section headers like "Abstract:###", "Review:###", "Decision:###"
    s = re.sub(r"(?im)^\s*(abstract|review|comment|decision)\s*[:#\-]*\s*", "", s).strip()
    # drop trailing " | OpenReview"
    s = re.sub(r"\s*\|\s*openreview\s*$", "", s, flags=re.I)
    return s

def canonical_title(t: str) -> str:
    return norm_ws(strip_prefixes(t)).lower()

def canonical_abstract(a: str) -> str:
    return norm_ws(strip_prefixes(a)).lower()

def abstract_key_short(a: str, n=240) -> str:
    return canonical_abstract(a)[:n]

def jaccard_tokens(a: str, b: str) -> float:
    A = set(re.findall(r"\w+", a.lower()))
    B = set(re.findall(r"\w+", b.lower()))
    if not A and not B: return 1.0
    if not A or not B:  return 0.0
    return len(A & B) / len(A | B)

def parse_decision(x):
    s = "" if x is None else str(x)
    s = s.strip()
    # direct labels
    if s.lower() in ("accept","reject"): return s.title()
    # typical lines: "Decision:###Reject", "Decision: Reject"
    m = re.search(r"(?im)^\s*#{0,3}\s*decision\s*[:#\-]*\s*(accept|reject)\b", s)
    if m: return m.group(1).title()
    # fallback: last bare Accept/Reject mention
    hits = [h.group(1).lower() for h in re.finditer(r"\b(accept|reject)\b", s, flags=re.I)]
    if hits: return "Accept" if hits[-1] == "accept" else "Reject"
    return None

def pick_pred_text(rec: dict) -> str:
    for k in PRED_TEXT_KEYS:
        v = rec.get(k)
        if isinstance(v, str) and v.strip():
            return v.strip()
    v = rec.get("raw")
    if isinstance(v, str) and v.strip():
        return v.strip()
    return ""

# ---------- Load test set (with your exact prefixes) ----------
assert Path(TEST_PARQUET).exists(), f"Missing {TEST_PARQUET}"
test_df = pd.read_parquet(TEST_PARQUET)

need = {"title", "abstract", "review", "paper_decision"}
missing = need - set(test_df.columns)
assert not missing, f"Test set missing columns: {missing}"

# Canonical keys
test_df["t_can"] = test_df["title"].apply(canonical_title)
test_df["a_can"] = test_df["abstract"].apply(canonical_abstract)
test_df["a_key"] = test_df["abstract"].apply(abstract_key_short)

# Multiple reviews per paper? Aggregate ground-truth decision by mode across rows:
def mode_decision(series):
    decs = [parse_decision(x) for x in series]
    decs = [d for d in decs if d in ("Accept","Reject")]
    if not decs: return None
    c = collections.Counter(decs)
    return c.most_common(1)[0][0]

gt = (
    test_df.groupby(["t_can","a_key"], as_index=False)
           .agg(title=("title","first"),
                abstract=("abstract","first"),
                gt_decision=("paper_decision", mode_decision))
)

# ---------- Load predictions and index them ----------
def load_jsonl(path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            try:
                rec = json.loads(line)
            except Exception:
                continue
            title = rec.get("title","")
            abstract = rec.get("abstract","")
            text = pick_pred_text(rec)
            # take decision from normalized field if present, otherwise parse from text
            dec = rec.get("parsed_decision") or rec.get("decision") or rec.get("paper_decision") or parse_decision(text)
            rows.append({
                "title": title,
                "abstract": abstract,
                "t_can": canonical_title(title),
                "a_can": canonical_abstract(abstract),
                "a_key": abstract_key_short(abstract),
                "text": text,
                "decision": parse_decision(dec)
            })
    return rows

def index_predictions(pred_rows):
    exact = {}  # (t_can, a_key) -> row with longest text
    by_title = collections.defaultdict(list)  # t_can -> [rows]
    for r in pred_rows:
        key = (r["t_can"], r["a_key"])
        if key not in exact or len(r["text"]) > len(exact[key]["text"]):
            exact[key] = r
        by_title[r["t_can"]].append(r)
    return exact, by_title

# Baseline
assert Path(BASELINE_JSONL).exists(), f"Missing {BASELINE_JSONL}"
base_rows = load_jsonl(BASELINE_JSONL)
base_exact, base_by_title = index_predictions(base_rows)

# Trained
assert Path(TRAINED_JSONL).exists(), f"Missing {TRAINED_JSONL}"
trn_rows = load_jsonl(TRAINED_JSONL)
trn_exact, trn_by_title = index_predictions(trn_rows)

# ---------- Alignment: exact (title + short-abstract), then fuzzy by title ----------
def align_one(t_can, a_key, a_can, exact_map, title_map, abs_jaccard_thresh=0.30):
    r = exact_map.get((t_can, a_key))
    if r: return r
    cands = title_map.get(t_can, [])
    if not cands: return None
    best, best_j = None, -1.0
    for c in cands:
        j = jaccard_tokens(a_can, c["a_can"])
        if j > best_j:
            best_j, best = j, c
    return best if best_j >= abs_jaccard_thresh else None

def align_block(gt_df, exact_map, title_map, prefix):
    found, text_col, dec_col = [], [], []
    for row in gt_df.itertuples(index=False):
        r = align_one(row.t_can, row.a_key, canonical_abstract(row.abstract), exact_map, title_map)
        if r is None:
            found.append(False); text_col.append(""); dec_col.append(None)
        else:
            found.append(True);  text_col.append(r["text"]); dec_col.append(r["decision"])
    gt_df[f"{prefix}_found"] = found
    gt_df[f"{prefix}_text"]  = text_col
    gt_df[f"{prefix}_decision"] = dec_col
    return gt_df

gt = align_block(gt, base_exact, base_by_title, "base")
gt = align_block(gt, trn_exact,  trn_by_title,  "trained")

n_base = int(gt["base_found"].sum())
n_trn  = int(gt["trained_found"].sum())
print(f"Aligned baseline for {n_base}/{len(gt)} papers; trained for {n_trn}/{len(gt)} papers.")

# Write some unmatched samples to inspect (helps tune thresholds if needed)
unmatched = gt[(~gt["base_found"]) & (~gt["trained_found"])][["title","abstract"]].head(20)
if len(unmatched):
    unmatched.to_csv(UNMATCHED_CSV, index=False)
    print(f"⚠️ Wrote {len(unmatched)} unmatched sample rows to {UNMATCHED_CSV}")

# ---------- Decision accuracy ----------
def decision_metrics(df_g, pred_col, gt_col="gt_decision"):
    sub = df_g[[pred_col, gt_col]].dropna()
    sub = sub[sub[pred_col].isin(["Accept","Reject"])]
    if len(sub)==0:
        return {"accuracy": None, "n": 0, "cm": {}}
    acc = float((sub[pred_col]==sub[gt_col]).mean())
    cm = collections.Counter(zip(sub[gt_col], sub[pred_col]))
    return {"accuracy": acc, "n": int(len(sub)), "cm": dict(cm)}

base_dec = decision_metrics(gt, "base_decision")
trn_dec  = decision_metrics(gt, "trained_decision")

def fmt(m):
    return "n/a" if m["accuracy"] is None else f"{m['accuracy']:.3f}"

print("\n=== Decision accuracy ===")
print(f"Baseline: acc={fmt(base_dec)}  n={base_dec['n']}  cm={base_dec['cm']}")
print(f"Trained : acc={fmt(trn_dec)}  n={trn_dec['n']}  cm={trn_dec['cm']}")


Aligned baseline for 2203/2203 papers; trained for 2203/2203 papers.

=== Decision accuracy ===
Baseline: acc=0.309  n=2203  cm={('Reject', 'Accept'): 1521, ('Accept', 'Accept'): 678, ('Reject', 'Reject'): 2, ('Accept', 'Reject'): 2}
Trained : acc=0.417  n=2196  cm={('Reject', 'Accept'): 1126, ('Reject', 'Reject'): 391, ('Accept', 'Reject'): 155, ('Accept', 'Accept'): 524}


In [ ]:
# === Paths (edit as needed) ===
TEST_PARQUET          = "test_2020.parquet"           # has columns: title, abstract, review, paper_decision
BASELINE_JSONL        = "BASELINE_llama32_test20_NORM.jsonl"  #  baseline outputs (normalized or raw)
TRAINED_JSONL         = "TRAINED_llama32_test20_raw_output_NORM.jsonl"   #  trained outputs (normalized or raw)
OUT_CSV               = "review_similarity_metrics.csv"     # per-paper metrics

import re, json, math, collections
from pathlib import Path
import pandas as pd
import numpy as np

# ---------- Helpers ----------
PRED_TEXT_KEYS = ["review","comment","raw","output","response","completion"]

def norm_ws(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "")).strip()

def strip_prefixes(s: str) -> str:
    s = str(s or "")
    s = re.sub(r"(?im)^\s*(abstract|review|comment|decision)\s*[:#\-]*\s*", "", s).strip()
    s = re.sub(r"\s*\|\s*openreview\s*$", "", s, flags=re.I)
    return s

def canonical_title(t: str) -> str:
    return norm_ws(strip_prefixes(t)).lower()

def canonical_abstract(a: str) -> str:
    return norm_ws(strip_prefixes(a)).lower()

def abstract_key_short(a: str, n=240) -> str:
    return canonical_abstract(a)[:n]

def pick_pred_text(rec: dict) -> str:
    for k in PRED_TEXT_KEYS:
        v = rec.get(k)
        if isinstance(v, str) and v.strip():
            return v.strip()
    return (rec.get("raw_original") or rec.get("raw") or "").strip()

def load_jsonl(path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            try:
                rec = json.loads(line)
            except Exception:
                continue
            title = rec.get("title","")
            abstract = rec.get("abstract","")
            text = pick_pred_text(rec)
            rows.append({
                "title": title,
                "abstract": abstract,
                "t_can": canonical_title(title),
                "a_can": canonical_abstract(abstract),
                "a_key": abstract_key_short(abstract),
                "text": text
            })
    return rows

def index_predictions(pred_rows):
    exact = {}  # (t_can, a_key) -> row with longest text
    by_title = collections.defaultdict(list)  # t_can -> [rows]
    for r in pred_rows:
        key = (r["t_can"], r["a_key"])
        if key not in exact or len(r["text"]) > len(exact[key]["text"]):
            exact[key] = r
        by_title[r["t_can"]].append(r)
    return exact, by_title

def jaccard_tokens(a: str, b: str) -> float:
    A = set(re.findall(r"\w+", (a or "").lower()))
    B = set(re.findall(r"\w+", (b or "").lower()))
    if not A and not B: return 1.0
    if not A or not B:  return 0.0
    return len(A & B) / len(A | B)

def align_one(t_can, a_key, a_can, exact_map, title_map, abs_jaccard_thresh=0.30):
    r = exact_map.get((t_can, a_key))
    if r: return r
    cands = title_map.get(t_can, [])
    if not cands: return None
    best, best_j = None, -1.0
    for c in cands:
        j = jaccard_tokens(a_can, c["a_can"])
        if j > best_j:
            best_j, best = j, c
    return best if best_j >= abs_jaccard_thresh else None

# ---------- Load ground truth ----------
assert Path(TEST_PARQUET).exists(), f"Missing {TEST_PARQUET}"
test_df = pd.read_parquet(TEST_PARQUET)

need = {"title","abstract","review","paper_decision"}
missing = need - set(test_df.columns)
assert not missing, f"Test set missing columns: {missing}"

test_df["t_can"] = test_df["title"].apply(canonical_title)
test_df["a_can"] = test_df["abstract"].apply(canonical_abstract)
test_df["a_key"] = test_df["abstract"].apply(abstract_key_short)

# group multiple human reviews per paper
gt_grouped = (
    test_df.groupby(["t_can","a_key"], as_index=False)
           .agg(title=("title","first"),
                abstract=("abstract","first"),
                human_reviews=("review", list))
)

# ---------- Load predictions ----------
assert Path(BASELINE_JSONL).exists(), f"Missing {BASELINE_JSONL}"
assert Path(TRAINED_JSONL).exists(), f"Missing {TRAINED_JSONL}"

base_rows = load_jsonl(BASELINE_JSONL)
trn_rows  = load_jsonl(TRAINED_JSONL)

base_exact, base_by_title = index_predictions(base_rows)
trn_exact,  trn_by_title  = index_predictions(trn_rows)

# align
aligned = []
for row in gt_grouped.itertuples(index=False):
    t_can, a_key = row.t_can, row.a_key
    a_can = canonical_abstract(row.abstract)

    b = align_one(t_can, a_key, a_can, base_exact, base_by_title)
    t = align_one(t_can, a_key, a_can, trn_exact, trn_by_title)

    if (b is None) and (t is None):
        continue
    aligned.append({
        "title": row.title,
        "abstract": row.abstract,
        "human_reviews": row.human_reviews,
        "baseline_text": (b or {}).get("text",""),
        "trained_text":  (t or {}).get("text",""),
        "has_base": b is not None,
        "has_trained": t is not None,
    })

print(f"Aligned papers: {len(aligned)} "
      f"(baseline for {sum(a['has_base'] for a in aligned)}, trained for {sum(a['has_trained'] for a in aligned)})")

# ---------- Similarity engines ----------
use_sbert = False
try:
    from sentence_transformers import SentenceTransformer, util
    sbert = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device="cuda")
    use_sbert = True
    print("SBERT loaded for semantic similarity (CPU).")
except Exception as e:
    print("SBERT not available, will compute token Jaccard only.", e)

def sbert_cos(a: str, b: str) -> float:
    ea = sbert.encode([a or ""], convert_to_tensor=True, normalize_embeddings=True)
    eb = sbert.encode([b or ""], convert_to_tensor=True, normalize_embeddings=True)
    return float((ea @ eb.T).cpu().numpy()[0,0])

# ---------- Compute per-paper metrics ----------
rows = []
for item in aligned:
    H = [norm_ws(strip_prefixes(x)) for x in item["human_reviews"] if isinstance(x, str) and x.strip()]
    if not H:
        continue
    btxt = norm_ws(strip_prefixes(item["baseline_text"]))
    ttxt = norm_ws(strip_prefixes(item["trained_text"]))

    # token-level similarity
    b_jacs = [jaccard_tokens(btxt, h) for h in H] if item["has_base"] else []
    t_jacs = [jaccard_tokens(ttxt, h) for h in H] if item["has_trained"] else []

    # semantic similarity
    if use_sbert:
        b_sims = [sbert_cos(btxt, h) for h in H] if item["has_base"] else []
        t_sims = [sbert_cos(ttxt, h) for h in H] if item["has_trained"] else []
    else:
        b_sims, t_sims = [], []

    def agg(v):
        return dict(mean=float(np.mean(v)) if v else np.nan,
                    max=float(np.max(v)) if v else np.nan)

    rows.append({
        "title": item["title"],
        "n_human_reviews": len(H),
        # Jaccard
        "baseline_jacc_mean": agg(b_jacs)["mean"],
        "baseline_jacc_max":  agg(b_jacs)["max"],
        "trained_jacc_mean":  agg(t_jacs)["mean"],
        "trained_jacc_max":   agg(t_jacs)["max"],
        # SBERT cosine
        "baseline_sbert_mean": agg(b_sims)["mean"],
        "baseline_sbert_max":  agg(b_sims)["max"],
        "trained_sbert_mean":  agg(t_sims)["mean"],
        "trained_sbert_max":   agg(t_sims)["max"],
    })

metrics_df = pd.DataFrame(rows)

def paired_summary(df, col_base, col_trn, label):
    sub = df[[col_base, col_trn]].dropna()
    if len(sub)==0:
        print(f"{label}: no paired data")
        return
    base_mean = sub[col_base].mean()
    trn_mean  = sub[col_trn].mean()
    delta     = (sub[col_trn] - sub[col_base]).mean()
    print(f"{label}:  baseline={base_mean:.4f}  trained={trn_mean:.4f}  Δ={delta:+.4f}  (n={len(sub)})")

print("\n=== Regression-style similarity metrics (higher is better) ===")
paired_summary(metrics_df, "baseline_jacc_mean", "trained_jacc_mean", "Token Jaccard (mean vs all human reviews)")
paired_summary(metrics_df, "baseline_jacc_max",  "trained_jacc_max",  "Token Jaccard (max over human reviews)")

if use_sbert:
    paired_summary(metrics_df, "baseline_sbert_mean", "trained_sbert_mean", "SBERT cosine (mean vs all)")
    paired_summary(metrics_df, "baseline_sbert_max",  "trained_sbert_max",  "SBERT cosine (max over human)")

# Save for your plots / further stats
metrics_df.to_csv(OUT_CSV, index=False)
print(f"\nSaved per-paper metrics to {OUT_CSV}")


Aligned papers: 2203 (baseline for 2203, trained for 2203)
SBERT loaded for semantic similarity (CPU).

=== Regression-style similarity metrics (higher is better) ===
Token Jaccard (mean vs all human reviews):  baseline=0.1300  trained=0.1457  Δ=+0.0157  (n=2203)
Token Jaccard (max over human reviews):  baseline=0.1480  trained=0.1633  Δ=+0.0154  (n=2203)
SBERT cosine (mean vs all):  baseline=0.6326  trained=0.5854  Δ=-0.0471  (n=2203)
SBERT cosine (max over human):  baseline=0.7046  trained=0.6482  Δ=-0.0564  (n=2203)

Saved per-paper metrics to review_similarity_metrics.csv


In [ ]:
# === Text similarity evaluation for strengths/weaknesses quality ===
# Metrics: ROUGE-1/2/L (F1), chrF, SBERT cosine, BERTScore (F1)
# Baseline vs Trained, with both pairwise-mean and max-over-human-per-paper aggregation.

# ------------------ imports & setup ------------------
import os, re, json, math, statistics, random
from pathlib import Path
import pandas as pd
from collections import defaultdict, Counter
from sacrebleu.metrics import CHRF as SacreCHRF


TEST_PATH          = "test_2020.parquet"           # has columns: title, abstract, review, paper_decision
BASELINE_JSONL        = "BASELINE_llama32_test20_NORM.jsonl"  #  baseline outputs (normalized or raw)
TRAINED_JSONL         = "TRAINED_llama32_test20_raw_output_NORM.jsonl"   #  trained outputs (normalized or raw)


import os, re, json, math, random, hashlib
from pathlib import Path
import pandas as pd
from collections import defaultdict

# --- light deps (CPU-safe) ---
def _ensure(pkg, pipname=None, import_as=None):
    try:
        __import__(import_as or pkg)
    except Exception:
        import subprocess, sys
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", pipname or pkg], check=True)
        __import__(import_as or pkg)

_ensure("sentence_transformers","sentence-transformers")
_ensure("rouge_score","rouge-score")
_ensure("sacrebleu","sacrebleu")

from sentence_transformers import SentenceTransformer, util
from rouge_score.rouge_scorer import RougeScorer
from sacrebleu.metrics import CHRF
import torch

random.seed(123)

# ---------- normalization & keying ----------
def normalize_ws(s:str) -> str:
    s = (s or "").replace("\r"," ").strip()
    s = re.sub(r"\s+", " ", s)
    return s

def strip_prefixes(s:str) -> str:
    s = s or ""
    return re.sub(r"^\s*(Abstract|Review)\s*[:#]*\s*", "", s, flags=re.I)

def strip_openreview_suffix(title:str) -> str:
    return re.sub(r"\s*\|\s*OpenReview\s*$", "", title or "", flags=re.I).strip()

def norm_title(s:str) -> str:
    return normalize_ws(strip_openreview_suffix(s))

def norm_abstract(s:str) -> str:
    return normalize_ws(strip_prefixes(s))

def make_rid(title:str, abstract:str) -> str:
    t = norm_title(title)
    a = norm_abstract(abstract)
    h = hashlib.sha1(); h.update(t.encode()); h.update(b"\x00"); h.update(a.encode())
    return h.hexdigest()

def load_test(path):
    p = Path(path)
    if p.suffix.lower() in {".parquet",".pq"}:
        df = pd.read_parquet(p)
    elif p.suffix.lower() in {".csv",".tsv"}:
        sep = "\t" if p.suffix.lower()==".tsv" else ","
        df = pd.read_csv(p, sep=sep)
    else:
        raise ValueError("TEST_PATH must be parquet/csv/tsv")
    for col in ["title","abstract","review"]:
        assert col in df.columns, f"Missing '{col}' in test set"
    df = df.copy()
    df["title_n"]    = df["title"].apply(norm_title)
    df["abstract_n"] = df["abstract"].apply(norm_abstract)
    df["rid"]        = [ make_rid(t,a) for t,a in zip(df["title_n"], df["abstract_n"]) ]
    return df

def read_jsonl(path):
    rows=[]
    with open(path,"r",encoding="utf-8") as f:
        for line in f:
            line=line.strip()
            if not line: continue
            try:
                rows.append(json.loads(line))
            except Exception:
                pass
    return pd.DataFrame(rows)

# ---------- extract strengths/weaknesses ----------
_POS_HINTS = ("strength","pro:","pros:","+ ","positive","novel","good","clear","useful","solid","well")
_NEG_HINTS = ("weak","con:","cons:","- ","negative","unclear","missing","limited","concern","issue","problem")

def _sent_split(text):
    parts = re.split(r'(?<=[\.\?\!])\s+|\n+', (text or "").strip())
    return [s.strip() for s in parts if s.strip()]

def extract_sw_from_text(text):
    s_list, w_list = [], []
    t = (text or "").replace("\r","")
    sec_s = re.search(r"(?:^|\n)\s*strengths?\s*[:#-]*\s*(.+?)(?:\n\s*weaknesses?[:#-]*|\Z)", t, flags=re.I|re.S)
    sec_w = re.search(r"(?:^|\n)\s*weaknesses?\s*[:#-]*\s*(.+)", t, flags=re.I|re.S)
    if sec_s:
        s_list = re.split(r'\n[-\*\+]\s+|\n+|\;\s+|(?<=[\.\!\?])\s+', sec_s.group(1))
    if sec_w:
        w_list = re.split(r'\n[-\*\+]\s+|\n+|\;\s+|(?<=[\.\!\?])\s+', sec_w.group(1))
    if not s_list or not w_list:
        sents = _sent_split(t)
        if not s_list:
            s_list = [s for s in sents if any(k in s.lower() for k in _POS_HINTS)]
        if not w_list:
            w_list = [s for s in sents if any(k in s.lower() for k in _NEG_HINTS)]
    def _clean_cap(lst, cap=8):
        out=[]
        for s in lst:
            s = re.sub(r'\s+',' ', s).strip().strip('\"\'•-+*#')
            if s and s not in out: out.append(s)
            if len(out)>=cap: break
        return out
    return _clean_cap(s_list), _clean_cap(w_list)

def parse_pred_row(r):
    # prefer explicit lists
    s_list = r.get("strengths", [])
    w_list = r.get("weaknesses", [])
    if not isinstance(s_list, list): s_list=[]
    if not isinstance(w_list, list): w_list=[]

    # fallbacks
    for key, dest in [("strengths_text","s"),("weaknesses_text","w")]:
        if isinstance(r.get(key), str) and r[key].strip():
            chunks = re.split(r'\n[-\*\+]\s+|\n+|\;\s+|(?<=[\.\!\?])\s+', r[key])
            chunks = [c.strip(" -+*#\t") for c in chunks if len(c.strip())>2]
            if dest=="s" and not s_list: s_list = chunks
            if dest=="w" and not w_list: w_list = chunks
    if (not s_list or not w_list):
        for key in ["review","comment","raw","raw_original"]:
            if isinstance(r.get(key), str) and r[key].strip():
                s2,w2 = extract_sw_from_text(r[key])
                if not s_list: s_list = s2
                if not w_list: w_list = w2
    return s_list[:4], w_list[:4]

def map_predictions(df_pred):
    """Return maps keyed by rid and by normalized title; each value: {'S':[], 'W':[], 'title_n','abstract_n','rid'}"""
    by_rid = {}
    by_title = {}
    for _,r in df_pred.iterrows():
        title = r.get("title") or r.get("paper_title") or ""
        abstract = r.get("abstract") or ""
        # if missing, try split from 'prompt'
        if not title and isinstance(r.get("prompt"), str):
            p = r["prompt"]
            try: title, abstract = p.split("\n\n",1)
            except: title, abstract = p, ""
        t_n = norm_title(title)
        a_n = norm_abstract(abstract)
        rid = r.get("rid") or make_rid(t_n, a_n)
        s_list, w_list = parse_pred_row(r.to_dict())
        rec = {"S":s_list, "W":w_list, "title_n":t_n, "abstract_n":a_n, "rid":rid}
        by_rid.setdefault(rid, rec)
        by_title.setdefault(t_n, rec)
    return by_rid, by_title

# ---------- text metrics ----------
ROUGE = RougeScorer(["rouge1","rouge2","rougeL"], use_stemmer=True)
CHRF = CHRF()

def rouge_all(ref, hyp):
    s = ROUGE.score(ref, hyp)
    return {k: v.fmeasure for k,v in s.items()}

def chrf(ref, hyp):
    return CHRF.sentence_score(hyp, [ref]).score/100.0

def jaccard(a,b):
    A = set([t for t in re.findall(r"\b\w+\b", (a or "").lower()) if len(t)>1])
    B = set([t for t in re.findall(r"\b\w+\b", (b or "").lower()) if len(t)>1])
    if not A and not B: return 1.0
    if not A or not B:  return 0.0
    return len(A&B)/len(A|B)

def concat_list(lst): return " ".join(lst or []).strip()

def soft_prf1(pred_items, gold_items, embedder, sim_thresh=0.60, batch_size=32):
    if not pred_items and not gold_items: return 1,1,1
    if not pred_items or not gold_items:  return 0,0,0
    with torch.no_grad():
        pe = embedder.encode(pred_items, convert_to_tensor=True, batch_size=batch_size, device="cuda", normalize_embeddings=True)
        ge = embedder.encode(gold_items, convert_to_tensor=True, batch_size=batch_size, device="cuda", normalize_embeddings=True)
        S = util.cos_sim(pe, ge).cpu().numpy()
    P,G = len(pred_items), len(gold_items)
    matched_p=set(); matched_g=set(); hits=0
    while True:
        best=(-1,None,None)
        for i in range(P):
            if i in matched_p: continue
            for j in range(G):
                if j in matched_g: continue
                if S[i,j]>best[0]: best=(S[i,j],i,j)
        if best[1] is None or best[0] < sim_thresh: break
        _,i,j = best
        matched_p.add(i); matched_g.add(j); hits += 1
    Pn = hits/max(P,1); Rn = hits/max(G,1)
    F1 = 0.0 if (Pn+Rn)==0 else 2*Pn*Rn/(Pn+Rn)
    return Pn,Rn,F1

# ---------- load & align ----------
df_test = load_test(TEST_PATH)            # has rid / title_n / abstract_n
df_base = read_jsonl(BASELINE_JSONL)
df_trn  = read_jsonl(TRAINED_JSONL)

base_by_rid, base_by_title = map_predictions(df_base)
trn_by_rid,  trn_by_title  = map_predictions(df_trn)

gold_rids = set(df_test["rid"].tolist())
keys_rid  = sorted(gold_rids & set(base_by_rid.keys()) & set(trn_by_rid.keys()))

if len(keys_rid)==0:
    # fallback: title-only join (warn)
    print("⚠️ No RID matches. Falling back to title-only alignment after normalization.")
    gold_titles = set(df_test["title_n"].tolist())
    keys_title  = sorted(gold_titles & set(base_by_title.keys()) & set(trn_by_title.keys()))
    if len(keys_title)==0:
        # show a quick diff to debug
        print("Still 0 matches. Sample gold titles:", df_test["title_n"].head(3).tolist())
        print("Sample baseline titles:", list(base_by_title.keys())[:3])
        print("Sample trained titles:", list(trn_by_title.keys())[:3])
        raise SystemExit("No aligned papers. Check normalization / fields in JSONLs.")
    ALIGN_MODE="title"
    keys = keys_title
else:
    ALIGN_MODE="rid"
    keys = keys_rid

print(f"Aligned papers ({ALIGN_MODE}): {len(keys)}")

# ---------- build gold S/W per paper ----------
gold_sw = defaultdict(lambda: {"S":[], "W":[]})
for _,r in df_test.iterrows():
    k = r["rid"] if ALIGN_MODE=="rid" else r["title_n"]
    s,w = extract_sw_from_text(r["review"])
    if s: gold_sw[k]["S"].append(s)
    if w: gold_sw[k]["W"].append(w)
# keep only with some gold
keys = [k for k in keys if (gold_sw[k]["S"] or gold_sw[k]["W"])]
print(f"Usable papers (have gold S/W): {len(keys)}")

# ---------- scoring ----------
sb_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device="cuda")
ROUGE_SCR = RougeScorer(["rouge1","rouge2","rougeL"], use_stemmer=True)
CHRF_MET = SacreCHRF() 

def score_side(side_map):  # side_map is base_by_* or trn_by_*
    rows=[]
    for k in keys:
        pred = side_map[k]
        pred_S, pred_W = pred["S"], pred["W"]
        gS_lists, gW_lists = gold_sw[k]["S"], gold_sw[k]["W"]
        gS_all = [s for L in gS_lists for s in L]
        gW_all = [w for L in gW_lists for w in L]

        PS,RS,FS = soft_prf1(pred_S, gS_all, sb_model, 0.60)
        PW,RW,FW = soft_prf1(pred_W, gW_all, sb_model, 0.60)

        predS_cat, predW_cat = concat_list(pred_S), concat_list(pred_W)
        gS_cands = [concat_list(L) for L in gS_lists] or [""]
        gW_cands = [concat_list(L) for L in gW_lists] or [""]

        # choose best gold by SBERT cosine
        def best_ref(cands, hypothesis):
            if cands==[""] or not hypothesis: return cands[0]
            with torch.no_grad():
                h = sb_model.encode([hypothesis], convert_to_tensor=True, normalize_embeddings=True)
                C = sb_model.encode(cands, convert_to_tensor=True, normalize_embeddings=True)
                idx = int(torch.argmax(util.cos_sim(h, C)).item())
            return cands[idx]

        gS_best = best_ref(gS_cands, predS_cat)
        gW_best = best_ref(gW_cands, predW_cat)

        rS = ROUGE_SCR.score(gS_best, predS_cat); rW = ROUGE_SCR.score(gW_best, predW_cat)
        cS = CHRF_MET.sentence_score(predS_cat, [gS_best]).score/100.0
        cW = CHRF_MET.sentence_score(predW_cat, [gW_best]).score/100.0
        jS = jaccard(gS_best, predS_cat); jW = jaccard(gW_best, predW_cat)

        rows.append({
            "S_P":PS,"S_R":RS,"S_F1":FS,"W_P":PW,"W_R":RW,"W_F1":FW,
            "S_r1":rS["rouge1"].fmeasure,"S_r2":rS["rouge2"].fmeasure,"S_rL":rS["rougeL"].fmeasure,
            "W_r1":rW["rouge1"].fmeasure,"W_r2":rW["rouge2"].fmeasure,"W_rL":rW["rougeL"].fmeasure,
            "S_chrf":cS,"W_chrf":cW,"S_jacc":jS,"W_jacc":jW
        })
    return pd.DataFrame(rows)

# pick the right prediction maps based on alignment mode
base_map = base_by_rid if ALIGN_MODE=="rid" else base_by_title
trn_map  = trn_by_rid  if ALIGN_MODE=="rid" else trn_by_title

base_df = score_side(base_map)
trn_df  = score_side(trn_map)

def summarize(df):
    return {k: round(df[k].mean(), 3) for k in df.columns}

print("\n=== Soft matching (SBERT) P/R/F1 ===")
print("Baseline:", {k:summarize(base_df).get(k) for k in ["S_P","S_R","S_F1","W_P","W_R","W_F1"]})
print("Trained :", {k:summarize(trn_df).get(k)  for k in ["S_P","S_R","S_F1","W_P","W_R","W_F1"]})

print("\n=== Text similarity (concat sections) ===")
for k in ["S_r1","S_r2","S_rL","W_r1","W_r2","W_rL","S_chrf","W_chrf","S_jacc","W_jacc"]:
    print(f"{k:7s}  base={summarize(base_df)[k]:.3f}   trn={summarize(trn_df)[k]:.3f}")


/home/mobs/.local/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


⚠️ No RID matches. Falling back to title-only alignment after normalization.
Aligned papers (title): 2203
Usable papers (have gold S/W): 2202

=== Soft matching (SBERT) P/R/F1 ===
Baseline: {'S_P': 0.15, 'S_R': 0.036, 'S_F1': 0.056, 'W_P': 0.052, 'W_R': 0.016, 'W_F1': 0.023}
Trained : {'S_P': 0.09, 'S_R': 0.02, 'S_F1': 0.03, 'W_P': 0.075, 'W_R': 0.014, 'W_F1': 0.022}

=== Text similarity (concat sections) ===
S_r1     base=0.182   trn=0.190
S_r2     base=0.023   trn=0.023
S_rL     base=0.124   trn=0.110
W_r1     base=0.149   trn=0.169
W_r2     base=0.017   trn=0.020
W_rL     base=0.098   trn=0.099
S_chrf   base=0.205   trn=0.217
W_chrf   base=0.193   trn=0.193
S_jacc   base=0.082   trn=0.093
W_jacc   base=0.071   trn=0.081


# KL anchoring to the base model

In [1]:
import os, re, json, math, time, gc, random
from pathlib import Path

# Make Transformers play nice in notebooks
os.environ["TRANSFORMERS_NO_APEX"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
torch.backends.cuda.matmul.allow_tf32 = True

import pandas as pd
from datasets import Dataset

from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, BitsAndBytesConfig, Trainer,
)
from transformers.trainer_callback import EarlyStoppingCallback
import torch.nn.functional as F
from peft import PeftModel, prepare_model_for_kbit_training

# ----------------- Paths & knobs -----------------
DATA_PATH   = Path("train_17_18_19.parquet")      # has columns: prompt, completion, year
MODEL_NAME  = "meta-llama/Llama-3.2-3B-Instruct"

# Continue from your best previous LoRA (change if needed)
RESUME_CKPT = Path("lora_json_outputs_es/checkpoint-1000")  # <-- existing LoRA ckpt dir
OUTPUT_DIR  = Path("lora_json_kl")                           # <-- new output dir for continued training

SEED = 42
random.seed(SEED); torch.manual_seed(SEED)

# Data/Tokenization
MAX_LEN               = 1408
BATCH_SIZE_PER_DEVICE = 4
GRAD_ACCUM_STEPS      = 8
LR                    = 1e-4
WARMUP_RATIO          = 0.03
WEIGHT_DECAY          = 0.1
TRAIN_EPOCHS_MAX      = 6
EVAL_STEPS            = 200
SAVE_STEPS            = 200
LOG_STEPS             = 50
PATIENCE              = 3

# KL regularizer
KL_LAMBDA      = 0.5      # weight of KL (try 0.2–1.0)
KL_MAX_TOKENS  = 256      # apply KL only on last N tokens per sequence

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype  = torch.bfloat16 if device == "cuda" else torch.float32
print("CUDA available:", torch.cuda.is_available())

assert DATA_PATH.exists(), f"Missing {DATA_PATH}"
assert RESUME_CKPT.exists(), f"Missing LoRA checkpoint at {RESUME_CKPT}"

# ----------------- Load data -----------------
raw_df = pd.read_parquet(DATA_PATH)
need = {"prompt","completion","year"}
missing = need - set(raw_df.columns)
assert not missing, f"Dataset missing columns: {missing}"

train_df = raw_df[raw_df["year"] <= 2020].copy()
eval_df  = raw_df[raw_df["year"] == 2021].copy()
if len(eval_df) == 0:
    # fallback small holdout
    eval_df = train_df.sample(frac=0.08, random_state=SEED)
    train_df = train_df.drop(eval_df.index)
    print(f"⚠️ No 2021 rows. Using {len(eval_df)} ({0.08*100:.0f}%) of train as validation.")
train_df = train_df.reset_index(drop=True)
eval_df  = eval_df.reset_index(drop=True)
print(f"Train rows: {len(train_df)} | Eval rows: {len(eval_df)}")

# ----------------- Build JSON targets from human reviews (same as your previous run) -----------------
POS_CUES = {"strong","novel","well","clear","solid","comprehensive","convincing","significant","improve","robust"}
NEG_CUES = {"weak","limited","unclear","missing","lacking","concern","issue","fail","noisy","incomplete","overfit","bias"}

def extract_decision(text: str):
    m = re.search(r"\bDecision\s*[:\-]?\s*(Accept|Reject)\b", text, re.I)
    return m.group(1).title() if m else None

def sentence_split(txt: str):
    parts = re.split(r'(?<=[\.\?\!])\s+', str(txt).strip())
    return [s.strip() for s in parts if s.strip()]

def bullets_from_text(text: str):
    lines = [l.strip(" -*+\t") for l in str(text).splitlines() if l.strip()]
    return [l for l in lines if len(l.split()) >= 3]

def pick_items(cands, cues, k=2):
    scored = []
    for s in cands:
        score = sum(1 for w in cues if re.search(rf"\b{re.escape(w)}\b", s, re.I))
        scored.append((score, -len(s), s))
    scored.sort(reverse=True)
    out = [s for _,__,s in scored[:k]]
    if len(out) < k:
        for s in cands:
            if s not in out and len(out) < k:
                out.append(s)
    return out[:k]

def to_target_json(review_text: str):
    dec = extract_decision(review_text)
    body = re.sub(r"\bDecision\s*[:\-]?\s*(Accept|Reject)\b.*", "", str(review_text), flags=re.I|re.S).strip()
    bullets = bullets_from_text(body)
    sents   = sentence_split(body)
    cands   = bullets if len(bullets) >= 4 else sents

    strengths  = pick_items(cands, POS_CUES, k=2)
    remaining  = [c for c in cands if c not in strengths]
    weaknesses = pick_items(remaining, NEG_CUES, k=2)

    # comment = 2-3 leftover sentences
    comment_sents = [s for s in sents if s not in strengths and s not in weaknesses]
    comment = " ".join(comment_sents[:3]).strip() or " ".join(sents[:3]).strip()

    if dec is None:
        pos_hits = sum(any(re.search(rf"\b{w}\b", s, re.I) for w in POS_CUES) for s in strengths)
        neg_hits = sum(any(re.search(rf"\b{w}\b", s, re.I) for w in NEG_CUES) for s in weaknesses)
        dec = "Reject" if neg_hits >= pos_hits else "Accept"

    def fill2(lst, filler):
        lst = [x for x in lst if x]
        while len(lst) < 2: lst.append(filler)
        return lst[:2]

    strengths  = fill2(strengths,  "Clear motivation and empirical support.")
    weaknesses = fill2(weaknesses, "Limited analysis and missing ablations.")

    return {
        "comment": comment[:1200],
        "strengths": [s[:400] for s in strengths],
        "weaknesses": [w[:400] for w in weaknesses],
        "decision": dec if dec in ("Accept","Reject") else "Reject",
    }

def build_json_rows(df: pd.DataFrame):
    rows = []
    for _, r in df.iterrows():
        ptxt = str(r["prompt"])
        try:
            title, abstract = ptxt.split("\n\n", 1)
        except Exception:
            title, abstract = ptxt, ""
        tgt = to_target_json(str(r["completion"]))
        sys_prompt = (
            "You are an ICLR peer reviewer. Return ONLY valid JSON with keys: "
            'comment, strengths, weaknesses, decision. Exactly two strengths and two weaknesses. '
            'Decision must be "Accept" or "Reject".'
        )
        train_prompt = (
            f"{sys_prompt}\n\nTITLE:\n{title}\n\nABSTRACT:\n{abstract}\n\nRespond with JSON only."
        )
        rows.append({"prompt": train_prompt, "target_json": json.dumps(tgt, ensure_ascii=False)})
    return pd.DataFrame(rows)

train_json_df = build_json_rows(train_df)
eval_json_df  = build_json_rows(eval_df)
print(f"Built {len(train_json_df)} train JSON rows | {len(eval_json_df)} eval JSON rows")

# ----------------- Tokenizer & masked tokenization -----------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tok_mask_batch(batch):
    in_ids, labels, attn = [], [], []
    for prompt, target_json in zip(batch["prompt"], batch["target_json"]):
        p_ids = tokenizer(prompt, add_special_tokens=False)["input_ids"]
        y_ids = tokenizer(target_json + tokenizer.eos_token, add_special_tokens=False)["input_ids"]

        max_prompt = MAX_LEN - len(y_ids)
        if max_prompt < 0:
            y_ids = y_ids[-(MAX_LEN-1):] + [tokenizer.eos_token_id]
            p_ids = []
        else:
            p_ids = p_ids[-max_prompt:]

        ids = p_ids + y_ids
        lab = [-100]*len(p_ids) + y_ids
        am  = [1]*len(ids)

        pad = MAX_LEN - len(ids)
        if pad > 0:
            ids += [tokenizer.pad_token_id]*pad
            lab += [-100]*pad
            am  += [0]*pad

        in_ids.append(ids); labels.append(lab); attn.append(am)
    return {"input_ids": in_ids, "labels": labels, "attention_mask": attn}

train_ds = Dataset.from_pandas(train_json_df).map(
    tok_mask_batch, batched=True, remove_columns=["prompt","target_json"]
)
eval_ds  = Dataset.from_pandas(eval_json_df).map(
    tok_mask_batch, batched=True, remove_columns=["prompt","target_json"]
)
print("Tokenized:", len(train_ds), len(eval_ds), train_ds.column_names)

# ----------------- Build teacher (frozen) & student (trainable LoRA) -----------------
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if device=="cuda" else torch.float32,
)

# Teacher (frozen)
teacher_base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, quantization_config=quant_config, device_map="auto", trust_remote_code=True
).eval()
for p in teacher_base.parameters():
    p.requires_grad_(False)

# Student base + prepare for k-bit training
student_base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, quantization_config=quant_config, device_map="auto", trust_remote_code=True
)
student_base = prepare_model_for_kbit_training(student_base)

# Load LoRA adapters as TRAINABLE (continue training)
student = PeftModel.from_pretrained(
    student_base,
    RESUME_CKPT,
    is_trainable=True,   # <-- critical
)
# Make sure grads can flow with checkpointing
if hasattr(student, "enable_input_require_grads"):
    student.enable_input_require_grads()
student.gradient_checkpointing_enable()
student.config.use_cache = False
student.train()

# Quick visibility
def print_trainable_parameters(m):
    total = sum(p.numel() for p in m.parameters())
    trainable = sum(p.numel() for p in m.parameters() if p.requires_grad)
    print(f"Trainable params: {trainable:,} / {total:,} ({100*trainable/total:.3f}%)")
print_trainable_parameters(student)

# ----------------- KL-anchored Trainer -----------------
class KLAnchoredTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, kl_lambda=0.5, kl_max_tokens=256, **kwargs):
        super().__init__(*args, **kwargs)
        assert teacher_model is not None, "teacher_model required"
        self.teacher = teacher_model
        self.kl_lambda = kl_lambda
        self.kl_max_tokens = kl_max_tokens

    @torch.no_grad()
    def _teacher_logits(self, input_ids, attention_mask):
        out = self.teacher(input_ids=input_ids, attention_mask=attention_mask, return_dict=True)
        return out.logits  # [B, T, V]

    def compute_loss(self, model, inputs, return_outputs=False, *args, **kwargs):
        # Standard LM loss
        outputs = model(**inputs, return_dict=True)
        loss = outputs.loss

        # KL on last N valid tokens only
        input_ids     = inputs["input_ids"]
        attention_mask= inputs.get("attention_mask", None)
        labels        = inputs.get("labels", None)

        if attention_mask is None:
            attention_mask = (input_ids != tokenizer.pad_token_id).long()

        with torch.no_grad():
            tea_logits = self._teacher_logits(input_ids, attention_mask)

        stu_logits = outputs.logits

        # softmax
        tea_prob = F.softmax(tea_logits.float(), dim=-1)
        stu_logp = F.log_softmax(stu_logits.float(), dim=-1)

        # positions to include: labels != -100 (i.e., target JSON part) AND last kl_max_tokens
        lengths = attention_mask.sum(dim=1)  # [B]
        T = stu_logits.size(1)
        ar = torch.arange(T, device=stu_logits.device).unsqueeze(0)  # [1, T]
        tail_mask = ar >= (lengths.unsqueeze(1) - self.kl_max_tokens).clamp_min(0)  # [B, T]
        label_mask = (labels != -100) if labels is not None else attention_mask.bool()
        use_mask = (tail_mask & label_mask).unsqueeze(-1)  # [B, T, 1]

        kl_token = F.kl_div(stu_logp, tea_prob, reduction='none')  # [B, T, V]
        kl_perpos = kl_token.sum(dim=-1)                           # [B, T]
        kl_selected = kl_perpos.masked_select(use_mask.squeeze(-1))

        if kl_selected.numel() > 0:
            kl_loss = kl_selected.mean()
            loss = loss + self.kl_lambda * kl_loss

        return (loss, outputs) if return_outputs else loss

# ----------------- TrainingArguments -----------------
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
args = TrainingArguments(
    output_dir=str(OUTPUT_DIR),
    num_train_epochs=TRAIN_EPOCHS_MAX,
    per_device_train_batch_size=BATCH_SIZE_PER_DEVICE,
    gradient_accumulation_steps=GRAD_ACCUM_STEPS,
    learning_rate=LR,
    warmup_ratio=WARMUP_RATIO,
    weight_decay=WEIGHT_DECAY,
    lr_scheduler_type="cosine",
    logging_steps=LOG_STEPS,
    save_strategy="steps",
    save_steps=SAVE_STEPS,
    eval_strategy="steps",
    eval_steps=EVAL_STEPS,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    bf16=(device=="cuda"),
    report_to="none",
    save_total_limit=3,
    seed=SEED,
    remove_unused_columns=False,
)

# ----------------- Build trainer -----------------
trainer = KLAnchoredTrainer(
    model=student,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    teacher_model=teacher_base.eval(),
    kl_lambda=KL_LAMBDA,
    kl_max_tokens=KL_MAX_TOKENS,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=PATIENCE)],
)

print("✅ Setup done. Ready to train with KL anchoring.")


CUDA available: True
⚠️ No 2021 rows. Using 723 (8%) of train as validation.
Train rows: 8320 | Eval rows: 723
Built 8320 train JSON rows | 723 eval JSON rows


Map:   0%|          | 0/8320 [00:00<?, ? examples/s]

Map:   0%|          | 0/723 [00:00<?, ? examples/s]

Tokenized: 8320 723 ['input_ids', 'labels', 'attention_mask']


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Trainable params: 24,313,856 / 1,827,777,536 (1.330%)
✅ Setup done. Ready to train with KL anchoring.


In [ ]:
# ============================================
# Cell 2: Train + save + quick sanity check
# ============================================
import math
from peft import PeftModel

print("🚀 Continuing training with KL anchoring from loaded LoRA weights…")
# IMPORTANT: we already loaded weights; we want a fresh optimizer/scheduler for this new run.
# DO NOT pass resume_from_checkpoint here.
trainer.train()
print("✅ Done. Best checkpoint:", trainer.state.best_model_checkpoint)

# Save updated adapters + tokenizer
trainer.model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

# Evaluate perplexity
metrics = trainer.evaluate()
if "eval_loss" in metrics:
    print(f"Eval loss: {metrics['eval_loss']:.4f} | Perplexity: {math.exp(metrics['eval_loss']):.2f}")

# -------- Quick generation sanity (3 samples) --------
def make_prompt(title, abstract):
    return (
        "You are an ICLR peer reviewer. Return ONLY valid JSON with keys: "
        'comment, strengths, weaknesses, decision. Exactly two strengths and two weaknesses. '
        'Decision must be \"Accept\" or \"Reject\".\n\n'
        f"TITLE:\n{title}\n\nABSTRACT:\n{abstract}\n\nRespond with JSON only."
    )

# Reload best for generation (student adapters over base)
del trainer
gc.collect(); 
if torch.cuda.is_available(): torch.cuda.empty_cache()

gen_base = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, quantization_config=BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32
    ),
    device_map="auto",
    trust_remote_code=True,
)
gen_model = PeftModel.from_pretrained(gen_base, OUTPUT_DIR).eval()

tok = tokenizer
eos_id = tok.eos_token_id[0] if isinstance(tok.eos_token_id,(list,tuple)) else tok.eos_token_id

import random
sampled = eval_df.sample(n=min(3, len(eval_df)), random_state=SEED if len(eval_df) >= 3 else random.randint(0,9999))
for _, r in sampled.iterrows():
    ptxt = str(r["prompt"])
    try:
        title, abstract = ptxt.split("\n\n", 1)
    except Exception:
        title, abstract = ptxt, ""
    prompt = make_prompt(title, abstract)
    enc = tok(prompt, return_tensors="pt", truncation=True, padding=False).to(gen_model.device)
    with torch.inference_mode():
        out = gen_model.generate(
            **enc,
            max_new_tokens=220,
            do_sample=False,            # deterministic sanity check
            eos_token_id=eos_id,
            pad_token_id=tok.pad_token_id,
            return_dict_in_generate=True,
        )
    full = tok.decode(out.sequences[0], skip_special_tokens=True)
    gen_only = full[len(prompt):].strip()
    print("\n--- SAMPLE OUTPUT ---\n", gen_only[:800])


🚀 Continuing training with KL anchoring from loaded LoRA weights…


Step,Training Loss,Validation Loss
200,17.792400,2.500527
400,17.071300,2.523604


In [3]:
# ===== Inspect a KL-anchored LoRA run + quick eval =====
import os, json, re, math, time, gc, random
from pathlib import Path
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
    TrainingArguments, Trainer
)
from peft import PeftModel

# ----------- CONFIG (edit these if your paths differ) -----------
DATA_PATH       = Path("train_17_18_19.parquet")          # train parquet with prompt/completion/year
KL_OUTPUT_DIR   = Path("lora_json_kl")                    # new KL-anchored adapters
OLD_OUTPUT_DIR  = Path("lora_json_outputs_es")            # previous adapters (optional; for comparison)
MAX_LEN         = 1408
SEED            = 42
MODEL_NAME      = "meta-llama/Llama-3.2-3B-Instruct"
BF16            = torch.cuda.is_available()
MAX_EVAL_SAMP   = 512      # keep small for a fast check
PER_DEV_EVAL_BS = 2

torch.backends.cuda.matmul.allow_tf32 = True
print("CUDA:", torch.cuda.is_available())

def read_state(path: Path):
    """Return trainer_state.json dict if present, else None."""
    ts = path / "trainer_state.json"
    if ts.exists():
        try:
            return json.loads(ts.read_text())
        except Exception:
            return None
    return None

def checkpoints(path: Path):
    """List checkpoint-* dirs sorted by step."""
    if not path.exists(): return []
    ck = []
    for p in path.iterdir():
        if p.is_dir() and p.name.startswith("checkpoint-"):
            try:
                step = int(p.name.split("-")[-1])
                ck.append((step, p))
            except: pass
    return [p for _, p in sorted(ck, key=lambda x: x[0])]

def state_summary(ts):
    if not ts: return {}
    best = ts.get("best_model_checkpoint")
    best_metric = ts.get("best_metric")
    gs = ts.get("global_step")
    ep = ts.get("epoch")
    # Finished runs usually end with a log_history entry containing train_runtime etc.
    log = ts.get("log_history", [])
    finished = any(("train_runtime" in e) for e in log[-5:])
    # pull a couple of recent eval losses
    eval_losses = [e["eval_loss"] for e in log if "eval_loss" in e]
    tail = eval_losses[-5:] if eval_losses else []
    return dict(best=best, best_metric=best_metric, global_step=gs, epoch=ep, finished=finished, eval_tail=tail)

def print_run_status(tag, out_dir: Path):
    print(f"\n=== {tag} | dir: {out_dir} ===")
    if not out_dir.exists():
        print("❌ not found"); return None
    ck = checkpoints(out_dir)
    print("checkpoints:", [int(c.name.split("-")[-1]) for c in ck][:6], "..." if len(ck) > 6 else "")
    # prefer top-level trainer_state; else take latest ckpt's state
    ts = read_state(out_dir)
    if not ts and ck:
        ts = read_state(ck[-1])
        print("(using latest checkpoint trainer_state.json)")
    if not ts:
        print("⚠️ no trainer_state.json found")
        return None
    ss = state_summary(ts)
    print("best_model_checkpoint:", ss.get("best"))
    print("best_metric (eval_loss):", ss.get("best_metric"))
    print("global_step:", ss.get("global_step"), "epoch:", ss.get("epoch"))
    print("finished flag (heuristic):", ss.get("finished"))
    if ss.get("eval_tail"):
        print("eval_loss tail:", [round(x,4) for x in ss["eval_tail"]])
    # presence of adapter files
    for probe in [out_dir, Path(ss["best"])] if ss.get("best") else [out_dir]:
        if probe and probe.exists():
            has_ad = (probe/"adapter_model.bin").exists() or (probe/"adapter_model.safetensors").exists()
            print(f"adapters in {probe.name}:", "✅" if has_ad else "⚠️ missing")
    return ss

# --------------- quick status report ---------------
kl_state = print_run_status("KL run", KL_OUTPUT_DIR)
old_state = print_run_status("Old run", OLD_OUTPUT_DIR) if OLD_OUTPUT_DIR.exists() else None

# ----------------- Build fast eval split -----------------
assert DATA_PATH.exists(), f"Missing {DATA_PATH}"
df = pd.read_parquet(DATA_PATH)
assert set(["prompt","completion","year"]).issubset(df.columns)

eval_df = df[df["year"] == 2021].copy()
if len(eval_df) == 0:
    # fallback: small holdout for a quick check (won't match your exact training split)
    eval_df = df.sample(frac=0.08, random_state=SEED).copy()
eval_df = eval_df.reset_index(drop=True)
if len(eval_df) > MAX_EVAL_SAMP:
    eval_df = eval_df.sample(n=MAX_EVAL_SAMP, random_state=SEED).reset_index(drop=True)
print(f"\nEval rows for quick check: {len(eval_df)}")

# ---------- rebuild JSON targets & tokenize like training (masked loss on target only) ----------
POS_CUES = {"strong","novel","well","clear","solid","comprehensive","convincing","significant","improve","robust"}
NEG_CUES = {"weak","limited","unclear","missing","lacking","concern","issue","fail","noisy","incomplete","overfit","bias"}

def extract_decision(text: str):
    m = re.search(r"\bDecision\s*[:\-]?\s*(Accept|Reject)\b", str(text), re.I)
    return m.group(1).title() if m else None

def sentence_split(txt: str):
    parts = re.split(r'(?<=[\.\?\!])\s+', str(txt).strip())
    return [s.strip() for s in parts if s.strip()]

def bullets_from_text(text: str):
    lines = [l.strip(" -*+\t") for l in str(text).splitlines() if l.strip()]
    return [l for l in lines if len(l.split()) >= 3]

def pick_items(cands, cues, k=2):
    scored = []
    for s in cands:
        score = sum(1 for w in cues if re.search(rf"\b{re.escape(w)}\b", s, re.I))
        scored.append((score, -len(s), s))
    scored.sort(reverse=True)
    out = [s for _,__,s in scored[:k]]
    if len(out) < k:
        for s in cands:
            if s not in out and len(out) < k:
                out.append(s)
    return out[:k]

def to_target_json(review_text: str):
    dec = extract_decision(review_text)
    body = re.sub(r"\bDecision\s*[:\-]?\s*(Accept|Reject)\b.*", "", str(review_text), flags=re.I|re.S).strip()
    bullets = bullets_from_text(body)
    sents   = sentence_split(body)
    cands   = bullets if len(bullets) >= 4 else sents
    strengths  = pick_items(cands, POS_CUES, k=2)
    remaining  = [c for c in cands if c not in strengths]
    weaknesses = pick_items(remaining, NEG_CUES, k=2)
    comment_sents = [s for s in sents if s not in strengths and s not in weaknesses]
    comment = " ".join(comment_sents[:3]).strip() or " ".join(sents[:3]).strip()
    if dec is None:
        pos_hits = sum(any(re.search(rf"\b{w}\b", s, re.I) for w in POS_CUES) for s in strengths)
        neg_hits = sum(any(re.search(rf"\b{w}\b", s, re.I) for w in NEG_CUES) for s in weaknesses)
        dec = "Reject" if neg_hits >= pos_hits else "Accept"
    def fill2(lst, filler):
        lst = [x for x in lst if x]
        while len(lst) < 2: lst.append(filler)
        return lst[:2]
    strengths  = fill2(strengths,  "Clear motivation and empirical support.")
    weaknesses = fill2(weaknesses, "Limited analysis and missing ablations.")
    return {
        "comment": comment[:1200],
        "strengths": [s[:400] for s in strengths],
        "weaknesses": [w[:400] for w in weaknesses],
        "decision": dec if dec in ("Accept","Reject") else "Reject",
    }

def build_eval_rows(df: pd.DataFrame):
    rows=[]
    for _, r in df.iterrows():
        ptxt = str(r["prompt"])
        try:
            title, abstract = ptxt.split("\n\n", 1)
        except Exception:
            title, abstract = ptxt, ""
        tgt = to_target_json(str(r["completion"]))
        sys_prompt = (
            "You are an ICLR peer reviewer. Return ONLY valid JSON with keys: "
            'comment, strengths, weaknesses, decision. Exactly two strengths and two weaknesses. '
            'Decision must be "Accept" or "Reject".'
        )
        prompt = f"{sys_prompt}\n\nTITLE:\n{title}\n\nABSTRACT:\n{abstract}\n\nRespond with JSON only."
        rows.append({"prompt": prompt, "target_json": json.dumps(tgt, ensure_ascii=False)})
    return pd.DataFrame(rows)

eval_json_df = build_eval_rows(eval_df)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tok_mask_batch(batch):
    in_ids, labels, attn = [], [], []
    for prompt, target_json in zip(batch["prompt"], batch["target_json"]):
        p_ids = tokenizer(prompt, add_special_tokens=False)["input_ids"]
        y_ids = tokenizer(target_json + tokenizer.eos_token, add_special_tokens=False)["input_ids"]
        max_prompt = MAX_LEN - len(y_ids)
        if max_prompt < 0:
            y_ids = y_ids[-(MAX_LEN-1):] + [tokenizer.eos_token_id]
            p_ids = []
        else:
            p_ids = p_ids[-max_prompt:]
        ids = p_ids + y_ids
        lab = [-100]*len(p_ids) + y_ids
        am  = [1]*len(ids)
        pad = MAX_LEN - len(ids)
        if pad > 0:
            ids += [tokenizer.pad_token_id]*pad
            lab += [-100]*pad
            am  += [0]*pad
        in_ids.append(ids); labels.append(lab); attn.append(am)
    return {"input_ids": in_ids, "labels": labels, "attention_mask": attn}

eval_ds = Dataset.from_pandas(eval_json_df).map(
    tok_mask_batch, batched=True, remove_columns=["prompt","target_json"]
)

# ----------------- Tiny eval helper -----------------
def eval_perplexity(adapter_dir: Path, tag: str):
    print(f"\n[Eval] {tag}: {adapter_dir}")
    if not adapter_dir.exists():
        print("  ❌ missing")
        return None
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    )
    base = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME, quantization_config=quant_config, device_map="auto", trust_remote_code=True
    )
    model = PeftModel.from_pretrained(base, adapter_dir).eval()
    args = TrainingArguments(
        output_dir=str(adapter_dir / "tmp_eval"),
        per_device_eval_batch_size=PER_DEV_EVAL_BS,
        dataloader_drop_last=False,
        report_to="none",
        bf16=BF16,
    )
    trainer = Trainer(model=model, args=args, eval_dataset=eval_ds)
    with torch.inference_mode():
        metrics = trainer.evaluate()
    loss = metrics.get("eval_loss", None)
    if loss is None:
        print("  ⚠️ no eval_loss")
        return None
    ppl = math.exp(loss)
    print(f"  eval_loss={loss:.4f}  perplexity={ppl:.2f}")
    # free VRAM
    del trainer, model, base
    gc.collect()
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    return dict(loss=loss, ppl=ppl)

# ----------------- Run evals -----------------
kl_metrics  = eval_perplexity(KL_OUTPUT_DIR, "KL-anchored LoRA")
old_metrics = eval_perplexity(OLD_OUTPUT_DIR, "Previous LoRA") if OLD_OUTPUT_DIR.exists() else None

# ----------------- Quick interpretation -----------------
print("\n=== Summary ===")
if kl_state:
    print("KL run finished (heuristic):", kl_state.get("finished"))
    print("Best ckpt:", kl_state.get("best"), "best eval_loss:", kl_state.get("best_metric"))
if old_state:
    print("Old run best:", old_state.get("best"), "best eval_loss:", old_state.get("best_metric"))
if kl_metrics and old_metrics:
    dd = kl_metrics["loss"] - old_metrics["loss"]
    print(f"Δ eval_loss (KL - Old) = {dd:+.4f}  (negative is better)")
elif kl_metrics:
    print("KL run eval done.")


CUDA: True

=== KL run | dir: lora_json_kl ===
checkpoints: [200, 600, 800] 
(using latest checkpoint trainer_state.json)
best_model_checkpoint: lora_json_kl/checkpoint-200
best_metric (eval_loss): 2.5005273818969727
global_step: 800 epoch: 3.076923076923077
finished flag (heuristic): False
eval_loss tail: [2.5005, 2.5236, 2.5612, 2.6002]
adapters in lora_json_kl: ✅
adapters in checkpoint-200: ✅

=== Old run | dir: lora_json_outputs_es ===
checkpoints: [400, 800, 1000] 
(using latest checkpoint trainer_state.json)
best_model_checkpoint: lora_json_outputs_es/checkpoint-400
best_metric (eval_loss): 2.2726945877075195
global_step: 1000 epoch: 3.8461538461538463
finished flag (heuristic): False
eval_loss tail: [2.3013, 2.2727, 2.273, 2.2834, 2.2843]
adapters in lora_json_outputs_es: ✅
adapters in checkpoint-400: ✅

Eval rows for quick check: 512


Map:   0%|          | 0/512 [00:00<?, ? examples/s]


[Eval] KL-anchored LoRA: lora_json_kl


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  eval_loss=2.3139  perplexity=10.11

[Eval] Previous LoRA: lora_json_outputs_es


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  eval_loss=2.2764  perplexity=9.74

=== Summary ===
KL run finished (heuristic): False
Best ckpt: lora_json_kl/checkpoint-200 best eval_loss: 2.5005273818969727
Old run best: lora_json_outputs_es/checkpoint-400 best eval_loss: 2.2726945877075195
Δ eval_loss (KL - Old) = +0.0375  (negative is better)
